# Setup

In [44]:
import pandas as pd
from urllib.request import Request, urlopen
import matplotlib.pyplot as plt
import numpy as np
import requests
import itertools

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Shifts

In [16]:
# load player data for same game as shift data above and convert it to a dictionary
req = Request('https://api.nhle.com/stats/rest/en/shiftcharts?cayenneExp=gameId=2020020001', 
              headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
page_json = json.loads(webpage)

In [17]:
shift_player_id = []
shift_player_name = []
shift_player_team = []
shift_period = []
shift_start = []
shift_end = []

In [18]:
shifts = page_json['data']
for i in range(len(shifts)):
    shift_player_id.append(shifts[i]['playerId'])
    player_first = shifts[i]['firstName']
    player_last = shifts[i]['lastName']
    shift_player_name.append(player_first + ' ' + player_last)
    shift_player_team.append(shifts[i]['teamAbbrev'])
    shift_period.append(shifts[i]['period'])
    shift_start.append(shifts[i]['startTime'])
    shift_end.append(shifts[i]['endTime'])

In [19]:
shift_df = pd.DataFrame({
    'playerID': shift_player_id,
    'player name': shift_player_name,
    'player team': shift_player_team,
    'period': shift_period,
    'shift start': shift_start,
    'shift end': shift_end
})

In [ ]:
shift_df

In [50]:
def event_during_shift(event_time, event_period, shift_start, shift_end, shift_period):
    # only compare times for shift start/end and event if they are in the same period
#     if event_period == shift_period:
        
    event_min = event_time[0:2]
    event_sec = event_time[3:5]
    shift_start_min = shift_start[0:2]
    shift_start_sec = shift_start[3:5]
    shift_end_min = shift_end[0:2]
    shift_end_sec = shift_end[3:5]

    # shift start and end and event all in same minute (ex. event 01:28, 01:20 - 01:59)
    if ((event_min >= shift_start_min) and (event_sec >= shift_start_sec) and
        (event_min <= shift_end_min) and (event_sec < shift_end_sec)):
        return True

    # shift start and event in same minute, shift end later (ex. event 01:28, 01:20 - 02:04)
    elif ((event_min >= shift_start_min) and (event_sec >= shift_start_sec) and (event_min < shift_end_min)):
        return True

    # shift end and event in same minute, shift start earlier (ex. event 1:28, 00:56 - 1:59)
    elif ((event_min > shift_start_min) and (event_min <= shift_end_min) and (event_sec < shift_end_sec)):
        return True

    # shift start and end and event all in separate minutes (ex. event 03:29, 02:58 - 04:05)
    elif ((event_min > shift_start_min) and (event_min < shift_end_min)):
        return True

    # OT goal exception
    elif (event_period == 4 and event_min == shift_end_min and event_sec == shift_end_sec):
        return True

    # shootout exception
#         elif (event_period == 5 and event_min == shift_start_min and 
#               event_sec == shift_start_sec and event_min == shift_end_min and event_sec == shift_end_sec):
#             return True
    elif event_period == 5:
        return False

    else:
        return False
    
    return False

# Get working scrape for single game

In [22]:
# load player data for same game as shift data above and convert it to a dictionary
req = Request('https://statsapi.web.nhl.com/api/v1/game/2020020001/feed/live', 
              headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
page_json = json.loads(webpage)

# nhl_events
game_data = page_json['gameData']
pxp = page_json['liveData']
events = pxp['plays']['allPlays']

In [66]:
# events df
event_id = []
game_id = []
player1_id = []
player1_name = []
player1_link = []
player1_type = []
player2_id = []
player2_name = []
player2_link = []
player2_type = []
primaryAssister_id = []
primaryAssister_name = []
primaryAssister_link = []
primaryAssister_type = []
primaryAssister_total = []
secondaryAssister_id = []
secondaryAssister_name = []
secondaryAssister_link = []
secondaryAssister_type = []
secondaryAssister_total = []
penaltyServer_id = []
penaltyServer_name = []
penaltyServer_link = []
penaltyServer_type = []
strength = []
gwg = []
emptyNet = []
scorer_total = []
event = []
eventCode = []
eventType = []
eventDescription = []
secondaryType = []
penaltySeverity = []
penaltyMinutes = []
eventIdx = []
period = []
periodType = []
periodTime = []
dateTime = []
awayGoals = []
homeGoals = []
xCoord = []
yCoord = []
team1_id = []
team1_name = []
team1_link = []
team1_tricode = []
away_p1 = []
away_p2 = []
away_p3 = []
away_p4 = []
away_p5 = []
away_p6 = []
home_p1 = []
home_p2 = []
home_p3 = []
home_p4 = []
home_p5 = []
home_p6 = []

# games df
game_link = []
season = []
game_type = []
game_status = []
away_team_id = []
away_team_link = []
away_team_name = []
away_team_abbrev = []
away_team_result = []
away_team_goals = []
away_coach = []
home_team_id = []
home_team_link = []
home_team_name = []
home_team_abbrev = []
home_team_result = []
home_team_goals = []
home_coach = []
game_length = []
game_start_time = []
game_end_time = []
referee1 = []
referee2 = []
linesman1 = []
linesman2 = []
        
# # teams df
# team_id = []
# team_name = []
# team_link = []
# team_abbrev = []
# home_arena = []
# home_city = []
# home_timezone = []
# division_id = []
# division_name = []
# conference_id = []
# conference_name = []
# franchise_id = []
# first_year = []
# team_site = []

In [67]:
# away_abbrev = game_data['teams']['away']['abbreviation']
# home_abbrev = game_data['teams']['home']['abbreviation']
# game_link.append(page_json['link'])
# season.append(page_json['gameData']['game']['season'])
# game_type.append(page_json['gameData']['game']['type'])
# game_status.append(page_json['gameData']['status']['abstractGameState'])
# away_team_id.append(page_json['gameData']['teams']['away']['id'])
# away_team_link.append(page_json['gameData']['teams']['away']['link'])
# away_team_name.append(page_json['gameData']['teams']['away']['name'])
# away_team_abbrev.append(away_abbrev)
# away_team_goals.append(page_json['liveData']['boxscore']['teams']['away']['teamStats']['teamSkaterStats']['goals'])
# away_coach.append(page_json['liveData']['boxscore']['teams']['away']['coaches'][0]['person']['fullName'])
# home_team_id.append(page_json['gameData']['teams']['home']['id'])
# home_team_link.append(page_json['gameData']['teams']['home']['link'])
# home_team_name.append(page_json['gameData']['teams']['home']['name'])
# home_team_abbrev.append(home_abbrev)
# home_team_goals.append(page_json['liveData']['boxscore']['teams']['home']['teamStats']['teamSkaterStats']['goals'])
# home_coach.append(page_json['liveData']['boxscore']['teams']['home']['coaches'][0]['person']['fullName'])
# game_length.append(events[-1]['about']['period'])
# game_start_time.append(page_json['gameData']['datetime']['dateTime'])
# game_end_time.append(page_json['gameData']['datetime']['endDateTime'])
# referee1.append(page_json['liveData']['boxscore']['officials'][0]['official']['fullName'])
# referee2.append(page_json['liveData']['boxscore']['officials'][1]['official']['fullName'])
# linesman1.append(page_json['liveData']['boxscore']['officials'][2]['official']['fullName'])
# linesman2.append(page_json['liveData']['boxscore']['officials'][3]['official']['fullName'])
# if away_team_goals > home_team_goals:
#     away_team_result.append('Win')
#     home_team_result.append('Loss')
# else:
#     away_team_result.append('Loss')
#     home_team_result.append('Win')
                 

for i in range(len(events)):
    
#     print(i)
    
    # add game_id to every row
    game_id.append(game_data['game']['pk'])
    
    # these keys exist for every event, whether hockey play or not
    event_result = events[i]['result']
    event_about = events[i]['about']
    
    eventIdx.append(event_about['eventIdx'])
    event_id.append(str(game_data['game']['pk']) + '-' + str(event_about['eventIdx'])) # primary key for nhl_events
    event.append(event_result['event'])
    eventCode.append(event_result['eventCode'])
    eventType.append(event_result['eventTypeId'])
    eventDescription.append(event_result['description'])
    period.append(event_about['period'])
    periodType.append(event_about['periodType'])
    periodTime.append(event_about['periodTime'])
    dateTime.append(event_about['dateTime'])
    awayGoals.append(event_about['goals']['away'])
    homeGoals.append(event_about['goals']['home'])
    
    # event is a hockey event (Shot, Goal, Hit, etc.)
    if 'players' in events[i]:
        
        player_info = events[i]['players']
        
        # event type is a Giveaway, Takeaway, or Missed Shot
        if len(player_info) == 1:
            player1_id.append(player_info[0]['player']['id'])
            player1_name.append(player_info[0]['player']['fullName'])
            player1_link.append(player_info[0]['player']['link'])
            player1_type.append(player_info[0]['playerType'])
            player2_id.append('N/A')
            player2_name.append('N/A')
            player2_link.append('N/A')
            player2_type.append('N/A')
            primaryAssister_id.append('N/A')
            primaryAssister_name.append('N/A')
            primaryAssister_link.append('N/A')
            primaryAssister_type.append('N/A')
            primaryAssister_total.append('N/A')
            secondaryAssister_id.append('N/A')
            secondaryAssister_name.append('N/A')
            secondaryAssister_link.append('N/A')
            secondaryAssister_type.append('N/A')
            secondaryAssister_total.append('N/A')
            penaltyServer_id.append('NULL')
            penaltyServer_name.append('NULL')
            penaltyServer_link.append('NULL')
            penaltyServer_type.append('NULL')
        # event type is not a Goal, Giveaway, Takeaway, or Missed Shot or it's a goal with no assists
        elif len(player_info) == 2:
            player1_id.append(player_info[0]['player']['id'])
            player1_name.append(player_info[0]['player']['fullName'])
            player1_link.append(player_info[0]['player']['link'])
            player1_type.append(player_info[0]['playerType'])
            player2_id.append(player_info[1]['player']['id'])
            player2_name.append(player_info[1]['player']['fullName'])
            player2_link.append(player_info[1]['player']['link'])
            player2_type.append(player_info[1]['playerType'])   
            primaryAssister_id.append('NULL')
            primaryAssister_name.append('NULL')
            primaryAssister_link.append('NULL')
            primaryAssister_type.append('NULL')
            primaryAssister_total.append('NULL')
            secondaryAssister_id.append('NULL')
            secondaryAssister_name.append('NULL')
            secondaryAssister_link.append('NULL')
            secondaryAssister_type.append('NULL')
            secondaryAssister_total.append('NULL')
            penaltyServer_id.append('NULL')
            penaltyServer_name.append('NULL')
            penaltyServer_link.append('NULL')
            penaltyServer_type.append('NULL')
        # event type is a Goal with one assist or a Penalty taken and served by different players
        elif len(player_info) == 3:
            player1_id.append(player_info[0]['player']['id'])
            player1_name.append(player_info[0]['player']['fullName'])
            player1_link.append(player_info[0]['player']['link'])
            player1_type.append(player_info[0]['playerType'])
            # event type is Goal
            if event_result['eventTypeId'] == 'GOAL':
                primaryAssister_id.append(player_info[1]['player']['id'])
                primaryAssister_name.append(player_info[1]['player']['fullName'])
                primaryAssister_link.append(player_info[1]['player']['link'])
                primaryAssister_type.append(player_info[1]['playerType'])
                primaryAssister_total.append(player_info[1]['seasonTotal'])
                player2_id.append(player_info[2]['player']['id'])
                player2_name.append(player_info[2]['player']['fullName'])
                player2_link.append(player_info[2]['player']['link'])
                player2_type.append(player_info[2]['playerType'])
                penaltyServer_id.append('NULL')
                penaltyServer_name.append('NULL')
                penaltyServer_link.append('NULL')
                penaltyServer_type.append('NULL')
            # event type is Penalty
            elif event_result['eventTypeId'] == 'PENALTY':
                player2_id.append(player_info[1]['player']['id'])
                player2_name.append(player_info[1]['player']['fullName'])
                player2_link.append(player_info[1]['player']['link'])
                player2_type.append(player_info[1]['playerType'])
                penaltyServer_id.append(player_info[2]['player']['id'])
                penaltyServer_name.append(player_info[2]['player']['fullName'])
                penaltyServer_link.append(player_info[2]['player']['link'])
                penaltyServer_type.append(player_info[2]['playerType'])
                primaryAssister_id.append('NULL')
                primaryAssister_name.append('NULL')
                primaryAssister_link.append('NULL')
                primaryAssister_type.append('NULL')
                primaryAssister_total.append('NULL')
        # event type is a Goal with two assists
        elif len(player_info) == 4:
            player1_id.append(player_info[0]['player']['id'])
            player1_name.append(player_info[0]['player']['fullName'])
            player1_link.append(player_info[0]['player']['link'])
            player1_type.append(player_info[0]['playerType'])
            player2_id.append(player_info[3]['player']['id'])
            player2_name.append(player_info[3]['player']['fullName'])
            player2_link.append(player_info[3]['player']['link'])
            player2_type.append(player_info[3]['playerType']) 
            primaryAssister_id.append(player_info[1]['player']['id'])
            primaryAssister_name.append(player_info[1]['player']['fullName'])
            primaryAssister_link.append(player_info[1]['player']['link'])
            primaryAssister_type.append(player_info[1]['playerType'])
            primaryAssister_total.append(player_info[1]['seasonTotal'])
            secondaryAssister_id.append(player_info[2]['player']['id'])
            secondaryAssister_name.append(player_info[2]['player']['fullName'])
            secondaryAssister_link.append(player_info[2]['player']['link'])
            secondaryAssister_type.append(player_info[2]['playerType'])
            secondaryAssister_total.append(player_info[2]['seasonTotal'])
            penaltyServer_id.append('NULL')
            penaltyServer_name.append('NULL')
            penaltyServer_link.append('NULL')
            penaltyServer_type.append('NULL')
        else:
            print('What the fuck')
        
        
        
        # event is a Goal, Shot, or Penalty
        if 'secondaryType' in event_result:
            secondaryType.append(event_result['secondaryType'])
        else:
            secondaryType.append('N/A')
         
        
        
        # event is a Penalty
        if event_result['eventTypeId'] == 'PENALTY':
            penaltySeverity.append(event_result['penaltySeverity'])
            penaltyMinutes.append(event_result['penaltyMinutes'])
        else:
            penaltySeverity.append('N/A')
            penaltyMinutes.append('N/A')
            
            
            
        # event is a Goal
        if event_result['eventTypeId'] == 'GOAL':
            strength.append(event_result['strength']['code'])
            gwg.append(event_result['gameWinningGoal'])
            # not sure why but some goal events are missing emptyNet key
            if 'emptyNet' in event_result:
                emptyNet.append(event_result['emptyNet'])
            else:
                emptyNet.append('N/A')
            scorer_total.append(player_info[0]['seasonTotal'])
        else:
            strength.append('N/A')
            gwg.append('N/A')
            emptyNet.append('N/A')
            scorer_total.append('N/A')
            
            
        # some events are missing coordinates, no idea why
        if events[i]['coordinates']:
            xCoord.append(events[i]['coordinates']['x'])
            yCoord.append(events[i]['coordinates']['y'])
        else:
            xCoord.append('Missing')
            yCoord.append('Missing')
        team1_id.append(events[i]['team']['id'])
        team1_name.append(events[i]['team']['name'])
        team1_link.append(events[i]['team']['link'])
        team1_tricode.append(events[i]['team']['triCode'])
        
#         # adding player shifts
#         away_players_on_ice = []
#         home_players_on_ice = []
#         for shift in range(len(shift_df)):
#             if event_during_shift(event_about['periodTime'], event_about['period'], shift_df['shift start'][shift],
#                                     shift_df['shift end'][shift], shift_df['period'][shift]) == True:
#                 if shift_df['player team'][shift] == away_abbrev:
#                     away_players_on_ice.append(shift_df['player name'][shift])
#                 elif shift_df['player team'][shift] == home_abbrev:
#                     home_players_on_ice.append(shift_df['player name'][shift])

        away_players_on_ice = []
        home_players_on_ice = []
        
        #             gen = (row for row in shift_df if shift_df['period'][row]==event_about['period'])
            
#             for x in gen:
#                 print(x)
        
        for shift in range(len(shift_df)):
            if event_about['period'] == shift_df['period'][shift]:
                if event_during_shift(event_about['periodTime'], event_about['period'], shift_df['shift start'][shift],
                                        shift_df['shift end'][shift], shift_df['period'][shift]) == True:
                    if shift_df['player team'][shift] == away_abbrev:
                        away_players_on_ice.append(shift_df['player name'][shift])
                    elif shift_df['player team'][shift] == home_abbrev:
                        home_players_on_ice.append(shift_df['player name'][shift])

#         away_players_on_ice = []
#         home_players_on_ice = []
        
#         for index, shift in shift_df.iterrows():
#             if event_during_shift(event_about['periodTime'], event_about['period'], shift_df['shift start'][shift],
#                                     shift_df['shift end'][shift], shift_df['period'][shift]) == True:
#                 if shift_df['player team'][shift] == away_abbrev:
#                     away_players_on_ice.append(shift_df['player name'][shift])
#                 elif shift_df['player team'][shift] == home_abbrev:
#                     home_players_on_ice.append(shift_df['player name'][shift])



#         def assign_players_to_event(event):
#             away_players_on_ice = []
#             home_players_on_ice = []
#             for shift in range(len(events)):
#                 if event_during_shift(event_about['periodTime'], event_about['period'], shift_df['shift start'][shift],
#                                         shift_df['shift end'][shift], shift_df['period'][shift]) == True:
#                     if shift_df['player team'][shift] == away_abbrev:
#                         away_players_on_ice.append(shift_df['player name'][shift])
#                     elif shift_df['player team'][shift] == home_abbrev:
#                         home_players_on_ice.append(shift_df['player name'][shift])
#             return away_players_on_ice, home_players_on_ice
        
# #         away_players_on_ice, home_players_on_ice = assign_players(event_about, shift_df)
#         test1 = event.apply(assign_players_to_event, axis=1)
                  
        away_p1.append(away_players_on_ice[0])
        away_p2.append(away_players_on_ice[1])
        away_p3.append(away_players_on_ice[2])
        away_p4.append(away_players_on_ice[3])
        if len(away_players_on_ice)>=5:
            away_p5.append(away_players_on_ice[4])
        else:
            away_p5.append('N/A')
        if len(away_players_on_ice)>=6:
            away_p6.append(away_players_on_ice[5])
        else:
            away_p6.append('N/A')
            
        home_p1.append(home_players_on_ice[0])
        home_p2.append(home_players_on_ice[1])
        home_p3.append(home_players_on_ice[2])
        home_p4.append(home_players_on_ice[3])
        if len(home_players_on_ice)>=5:
            home_p5.append(home_players_on_ice[4])
        else:
            home_p5.append('N/A')
        if len(home_players_on_ice)>=6:
            home_p6.append(home_players_on_ice[5])
        else:
            home_p6.append('N/A')
        
    # event is a non-hockey event (Game start/end, Period start/end, Stoppage, etc.)
    else:
        player1_id.append('N/A')
        player1_name.append('N/A')
        player1_link.append('N/A')
        player1_type.append('N/A')
        player2_id.append('N/A')
        player2_name.append('N/A')
        player2_link.append('N/A')
        player2_type.append('N/A')
        primaryAssister_id.append('N/A')
        primaryAssister_name.append('N/A')
        primaryAssister_link.append('N/A')
        primaryAssister_type.append('N/A')
        primaryAssister_total.append('N/A')
        secondaryAssister_id.append('N/A')
        secondaryAssister_name.append('N/A')
        secondaryAssister_link.append('N/A')
        secondaryAssister_type.append('N/A')
        secondaryAssister_total.append('N/A')
        penaltyServer_id.append('NULL')
        penaltyServer_name.append('NULL')
        penaltyServer_link.append('NULL')
        penaltyServer_type.append('NULL')
        strength.append('N/A')
        gwg.append('N/A')
        emptyNet.append('N/A')
        scorer_total.append('N/A')
        secondaryType.append('N/A')
        penaltySeverity.append('N/A')
        penaltyMinutes.append('N/A')
        xCoord.append('N/A')
        yCoord.append('N/A')
        team1_id.append('N/A')
        team1_name.append('N/A')
        team1_link.append('N/A')
        team1_tricode.append('N/A')
        away_p1.append('N/A')
        away_p2.append('N/A')
        away_p3.append('N/A')
        away_p4.append('N/A')
        away_p5.append('N/A')
        away_p6.append('N/A')
        home_p1.append('N/A')
        home_p2.append('N/A')
        home_p3.append('N/A')
        home_p4.append('N/A')
        home_p5.append('N/A')
        home_p6.append('N/A')

In [68]:
games_df = pd.DataFrame({
    
    'game_id': game_id[0],
    'game_link': game_link,
    'season': season,
    'game_type': game_type,
    'game_status': game_status,      
    'away_team_id': away_team_id,
    'away_team_link': away_team_link,
    'away_team_name': away_team_name,
    'away_team_abbrev': away_team_abbrev,
    'away_team_result': away_team_result,
    'away_team_goals': away_team_goals,
    'away_coach': away_coach,
    'home_team_id': home_team_id,
    'home_team_link': home_team_link,
    'home_team_name': home_team_name,
    'home_team_abbrev': home_team_abbrev,
    'home_team_result': home_team_result, # if home_team_goals < away_team goals
    'home_team_goals': home_team_goals,
    'home_coach': home_coach,
    'game_length': game_length,
    'game_start_time': game_start_time,
    'game_end_time': game_end_time,
    'referee1': referee1,
    'referee2': referee2,
    'linesman1': linesman1,
    'linesman2': linesman2
    
})

In [87]:
games_df

,game_id,game_link,season,game_type,game_status,away_team_id,away_team_link,away_team_name,away_team_abbrev,away_team_result,away_team_goals,away_coach,home_team_id,home_team_link,home_team_name,home_team_abbrev,home_team_result,home_team_goals,home_coach,game_length,game_start_time,game_end_time,referee1,referee2,linesman1,linesman2
0,2020020001,/api/v1/game/2020020001/feed/live,20202021,R,Final,5,/api/v1/teams/5,Pittsburgh Penguins,PIT,Loss,3,Mike Sullivan,4,/api/v1/teams/4,Philadelphia Flyers,PHI,Win,6,Alain Vigneault,3,2021-01-13T22:30:00Z,2021-01-14T00:53:50Z,Wes McCauley,Kelly Sutherland,Andrew Smith,Tyson Baker


In [69]:
# check games table lengths
for col in games_df.columns:
    print(col, len(games_df[col]))

game_id 0
game_link 0
season 0
game_type 0
game_status 0
away_team_id 0
away_team_link 0
away_team_name 0
away_team_abbrev 0
away_team_result 0
away_team_goals 0
away_coach 0
home_team_id 0
home_team_link 0
home_team_name 0
home_team_abbrev 0
home_team_result 0
home_team_goals 0
home_coach 0
game_length 0
game_start_time 0
game_end_time 0
referee1 0
referee2 0
linesman1 0
linesman2 0


In [59]:
### change all to underscore

events_df = pd.DataFrame({
    'event_id': event_id,
    'game_id': game_id,
    'player1_id': player1_id,
    'player1_name': player1_name,
    'player1_link': player1_link,
    'player1_type': player1_type,
    'player2_id': player2_id,
    'player2_name': player2_name,
    'player2_link': player2_link,
    'player2_type': player2_type,
    'event': event,
    'eventCode': eventCode,
    'eventType': eventType,
    'eventDescription': eventDescription,
    'secondaryType': secondaryType,
    'primaryAssister_id': primaryAssister_id,
    'primaryAssister_name': primaryAssister_name,
    'primaryAssister_link': primaryAssister_link,
    'primaryAssister_type': primaryAssister_type,
    'primaryAssister_total': primaryAssister_total,
    'secondaryAssister_id': secondaryAssister_id,
    'secondaryAssister_name': secondaryAssister_name,
    'secondaryAssister_link': secondaryAssister_link,
    'secondaryAssister_type': secondaryAssister_type,
    'secondaryAssister_total': secondaryAssister_total,
    'penaltyServer_id': penaltyServer_id,
    'penaltyServer_name': penaltyServer_name,
    'penaltyServer_link': penaltyServer_link,
    'penaltyServer_type': penaltyServer_type,
    'strength': strength,
    'gwg': gwg,
    'emptyNet': emptyNet,
    'scorer_total': scorer_total,
    'penaltySeverity': penaltySeverity,
    'penaltyMinutes': penaltyMinutes,
    'eventIdx': eventIdx,
    'period': period,
    'periodType': periodType,
    'periodTime': periodTime,
    'dateTime': dateTime,
    'awayGoals': awayGoals,
    'homeGoals': homeGoals,
    'xCoord': xCoord,
    'yCoord': yCoord,
    'team1_id': team1_id,
    'team1_name': team1_name,
    'team1_link': team1_link,
    'team1_tricode': team1_tricode,
    'away p1': away_p1,
    'away p2': away_p2,
    'away p3': away_p3,
    'away p4': away_p4,
    'away p5': away_p5,
    'away p6': away_p6,
    'home p1': home_p1,
    'home p2': home_p2,
    'home p3': home_p3,
    'home p4': home_p4,
    'home p5': home_p5,
    'home p6': home_p6
})

NameError: name 'penaltyServer_id' is not defined

In [62]:
# check events table lengths
for col in events_df.columns:
    print(col, len(events_df[col]))

event_id 584
game_id 584
player1_id 584
player1_name 584
player1_link 584
player1_type 584
player2_id 584
player2_name 584
player2_link 584
player2_type 584
event 584
eventCode 584
eventType 584
eventDescription 584
secondaryType 584
primaryAssister_id 584
primaryAssister_name 584
primaryAssister_link 584
primaryAssister_type 584
primaryAssister_total 584
secondaryAssister_id 584
secondaryAssister_name 584
secondaryAssister_link 584
secondaryAssister_type 584
secondaryAssister_total 584
strength 584
gwg 584
emptyNet 584
scorer_total 584
penaltySeverity 584
penaltyMinutes 584
eventIdx 584
period 584
periodType 584
periodTime 584
dateTime 584
awayGoals 584
homeGoals 584
xCoord 584
yCoord 584
team1_id 584
team1_name 584
team1_link 584
team1_tricode 584
away p1 584
away p2 584
away p3 584
away p4 584
away p5 584
away p6 584
home p1 584
home p2 584
home p3 584
home p4 584
home p5 584
home p6 584


In [28]:
events_df[30:50]

,event_id,game_id,player1_id,player1_name,player1_link,player1_type,player2_id,player2_name,player2_link,player2_type,event,eventCode,eventType,eventDescription,secondaryType,primaryAssister_id,primaryAssister_name,primaryAssister_link,primaryAssister_type,primaryAssister_total,secondaryAssister_id,secondaryAssister_name,secondaryAssister_link,secondaryAssister_type,secondaryAssister_total,strength,gwg,emptyNet,scorer_total,penaltySeverity,penaltyMinutes,eventIdx,period,periodType,periodTime,dateTime,awayGoals,homeGoals,xCoord,yCoord,team1_id,team1_name,team1_link,team1_tricode,away p1,away p2,away p3,away p4,away p5,away p6,home p1,home p2,home p3,home p4,home p5,home p6
30,2020020001-31,2020020001,8478067,Oskar Lindblom,/api/v1/people/8478067,Blocker,8479293,Brandon Tanev,/api/v1/people/8479293,Shooter,Blocked Shot,PHI21,BLOCKED_SHOT,Brandon Tanev shot blocked shot by Oskar Lindblom,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,31,1,REGULAR,04:30,2021-01-13T22:39:09Z,0,0,57.0,22.0,4,Philadelphia Flyers,/api/v1/teams/4,PHI,Mark Jankowski,Cody Ceci,Tristan Jarry,Marcus Pettersson,Sam Lafferty,Brandon Tanev,Sean Couturier,Erik Gustafsson,Robert Hagg,Oskar Lindblom,Travis Konecny,Carter Hart
31,2020020001-30,2020020001,8478043,Sam Lafferty,/api/v1/people/8478043,Shooter,8479394,Carter Hart,/api/v1/people/8479394,Goalie,Shot,PHI67,SHOT,Sam Lafferty Wrist Shot saved by Carter Hart,Wrist Shot,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,30,1,REGULAR,04:30,2021-01-13T22:39:09Z,0,0,59.0,-35.0,5,Pittsburgh Penguins,/api/v1/teams/5,PIT,Mark Jankowski,Cody Ceci,Tristan Jarry,Marcus Pettersson,Sam Lafferty,Brandon Tanev,Sean Couturier,Erik Gustafsson,Robert Hagg,Oskar Lindblom,Travis Konecny,Carter Hart
32,2020020001-32,2020020001,8478439,Travis Konecny,/api/v1/people/8478439,Shooter,8477465,Tristan Jarry,/api/v1/people/8477465,Goalie,Shot,PHI68,SHOT,Travis Konecny Wrist Shot saved by Tristan Jarry,Wrist Shot,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,32,1,REGULAR,04:39,2021-01-13T22:39:18Z,0,0,-59.0,-28.0,4,Philadelphia Flyers,/api/v1/teams/4,PHI,Mark Jankowski,Cody Ceci,Tristan Jarry,Marcus Pettersson,Sam Lafferty,Brandon Tanev,Sean Couturier,Erik Gustafsson,Robert Hagg,Oskar Lindblom,Travis Konecny,Carter Hart
33,2020020001-33,2020020001,8479293,Brandon Tanev,/api/v1/people/8479293,Hitter,8474037,James van Riemsdyk,/api/v1/people/8474037,Hittee,Hit,PHI69,HIT,Brandon Tanev hit James van Riemsdyk,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,33,1,REGULAR,04:54,2021-01-13T22:39:33Z,0,0,55.0,42.0,5,Pittsburgh Penguins,/api/v1/teams/5,PIT,Mark Jankowski,Mike Matheson,Tristan Jarry,Jared McCann,John Marino,Brandon Tanev,Justin Braun,James van Riemsdyk,Jakub Voracek,Ivan Provorov,Carter Hart,Nolan Patrick
34,2020020001-35,2020020001,8476873,Mark Jankowski,/api/v1/people/8476873,Scorer,8479394,Carter Hart,/api/v1/people/8479394,Goalie,Goal,PHI70,GOAL,"Mark Jankowski (1) Snap Shot, assists: Brandon...",Snap Shot,8479293,Brandon Tanev,/api/v1/people/8479293,Assist,1,8477955,Jared McCann,/api/v1/people/8477955,Assist,1,EVEN,False,False,1,N/A,N/A,35,1,REGULAR,05:12,2021-01-13T22:39:53Z,1,0,81.0,-5.0,5,Pittsburgh Penguins,/api/v1/teams/5,PIT,Sidney Crosby,Mike Matheson,Jake Guentzel,Tristan Jarry,John Marino,Evan Rodrigues,Scott Laughton,Michael Raffl,Travis Sanheim,Nicolas Aube-Kubel,Philippe Myers,Carter Hart
35,2020020001-34,2020020001,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,Stoppage,PHI206,STOP,Goalie Stopped,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,34,1,REGULAR,05:12,2021-01-13T22:39:53Z,1,0,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
36,2020020001-36,2020020001,8476872,Scott Laughton,/api/v1/people/8476872,Winner,8471675,Sidney Crosby,/api/v1/people/8471675,Loser,Faceoff,PHI23,FACEOFF,Scott Laughton faceoff won against Sidney Crosby,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,36,1,REGULAR,

In [ ]:
teams_df = pd.DataFrame({
    
    'team_id': team_id,
    'team_name': team_name,
    'team_link': team_link,
    'team_abbrev': team_abbrev,
    'home_arena': home_arena,
    'home_city': home_city,
    'home_timezone': home_timezone,
    'division_id': division_id,
    'division_name': division_name,
    'conference_id': conference_id,
    'conference_name': conference_name,
    'franchise_id': franchise_id,
    'first_year': first_year,
    'team_site': team_site
    
})

In [ ]:
"""
-Dataframe of players with basic info (player_id)
-Dataframe for each player with all of their games and stats for each (game_id)
-Dataframe for each team of all their games and stats for each (game_id)
-Dataframe for each player/team with all of their events


-Could make those last 3 functions, enter player and it calculates
"""

## DF Troubleshooting

In [ ]:
print(len(player1_id))
print(len(player1_name))
print(len(player1_link))
print(len(player1_type))
print(len(player2_id))
print(len(player2_name))
print(len(player2_link))
print(len(player2_type))
print(len(primaryAssister_id))
print(len(primaryAssister_name))
print(len(primaryAssister_link))
print(len(primaryAssister_type))
print(len(primaryAssister_total))
print(len(secondaryAssister_id))
print(len(secondaryAssister_name))
print(len(secondaryAssister_link))
print(len(secondaryAssister_type))
print(len(secondaryAssister_total))
print(len(strength))
print(len(gwg))
print(len(emptyNet))
print(len(scorer_total))
print(len(event))
print(len(eventCode))
print(len(eventType))
print(len(eventDescription))
print(len(secondaryType))
print(len(penaltySeverity))
print(len(penaltyMinutes))
print(len(eventIdx))
#         eventID = []
print(len(period))
print(len(periodType))
print(len(periodTime))
print(len(dateTime))
print(len(awayGoals))
print(len(homeGoals))
print(len(xCoord))
print(len(yCoord))
print(len(team1_id))
print(len(team1_name))
print(len(team1_link))
print(len(team1_tricode))

# 2019-2020 Regular Season

In [56]:
# create lists to store events for the entire season, to be converted to Dataframe
event_id = []
game_id = []
player1_id = []
player1_name = []
player1_link = []
player1_type = []
player2_id = []
player2_name = []
player2_link = []
player2_type = []
primaryAssister_id = []
primaryAssister_name = []
primaryAssister_link = []
primaryAssister_type = []
primaryAssister_total = []
secondaryAssister_id = []
secondaryAssister_name = []
secondaryAssister_link = []
secondaryAssister_type = []
secondaryAssister_total = []
penaltyServer_id = []
penaltyServer_name = []
penaltyServer_link = []
penaltyServer_type = []
strength = []
gwg = []
emptyNet = []
scorer_total = []
event = []
eventCode = []
eventType = []
eventDescription = []
secondaryType = []
penaltySeverity = []
penaltyMinutes = []
eventIdx = []
period = []
periodType = []
periodTime = []
dateTime = []
awayGoals = []
homeGoals = []
xCoord = []
yCoord = []
team1_id = []
team1_name = []
team1_link = []
team1_tricode = []
away_p1 = []
away_p2 = []
away_p3 = []
away_p4 = []
away_p5 = []
away_p6 = []
home_p1 = []
home_p2 = []
home_p3 = []
home_p4 = []
home_p5 = []
home_p6 = []

# 2019-2020 regular season had 1271 total games
root = '201902'
for game in range(1, 10): #1,1272
    if game<10:
        code = root+'000'+str(game)
    elif game<100:
        code = root+'00'+str(game)
    elif game<1000:
        code = root+'0'+str(game)
    else:
        code = root+str(game)
        
#     print(code)
    
    url = 'https://statsapi.web.nhl.com/api/v1/game/'+str(code)+'/feed/live'
    
    # access page and pull relevant JSON
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    page_json = json.loads(webpage)
    game_data = page_json['gameData']
    events = page_json['liveData']['plays']['allPlays']
    
    # get away and home team abbreviations
    away_abbrev = game_data['teams']['away']['abbreviation']
    home_abbrev = game_data['teams']['home']['abbreviation']
    
    # access corresponding shift data and build dataframe
    req = Request('https://api.nhle.com/stats/rest/en/shiftcharts?cayenneExp=gameId='+str(code), 
              headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    page_json = json.loads(webpage)
    shift_player_id = []
    shift_player_name = []
    shift_player_team = []
    shift_period = []
    shift_start = []
    shift_end = []
    shifts = page_json['data']
    for i in range(len(shifts)):
        shift_player_id.append(shifts[i]['playerId'])
        player_first = shifts[i]['firstName']
        player_last = shifts[i]['lastName']
        shift_player_name.append(player_first + ' ' + player_last)
        shift_player_team.append(shifts[i]['teamAbbrev'])
        shift_period.append(shifts[i]['period'])
        shift_start.append(shifts[i]['startTime'])
        shift_end.append(shifts[i]['endTime'])
    shift_df = pd.DataFrame({
        'playerID': shift_player_id,
        'player name': shift_player_name,
        'player team': shift_player_team,
        'period': shift_period,
        'shift start': shift_start,
        'shift end': shift_end
    })
    
    # iterate through all events and store as a row
    for i in range(len(events)):
        
#         print(i)
    
        # add game_id to every row
        game_id.append(game_data['game']['pk'])

        # these keys exist for every event, whether hockey play or not
        event_result = events[i]['result']
        event_about = events[i]['about']
    
        # event is a hockey event (Shot, Goal, Hit, etc.)
        if 'players' in events[i]:

            player_info = events[i]['players']

            # event type is a Giveaway, Takeaway, or Missed Shot
            if len(player_info) == 1:
                player1_id.append(player_info[0]['player']['id'])
                player1_name.append(player_info[0]['player']['fullName'])
                player1_link.append(player_info[0]['player']['link'])
                player1_type.append(player_info[0]['playerType'])
                player2_id.append('N/A')
                player2_name.append('N/A')
                player2_link.append('N/A')
                player2_type.append('N/A')
                primaryAssister_id.append('N/A')
                primaryAssister_name.append('N/A')
                primaryAssister_link.append('N/A')
                primaryAssister_type.append('N/A')
                primaryAssister_total.append('N/A')
                secondaryAssister_id.append('N/A')
                secondaryAssister_name.append('N/A')
                secondaryAssister_link.append('N/A')
                secondaryAssister_type.append('N/A')
                secondaryAssister_total.append('N/A')
                penaltyServer_id.append('N/A')
                penaltyServer_name.append('N/A')
                penaltyServer_link.append('N/A')
                penaltyServer_type.append('N/A')
            # event type is not a Goal, Giveaway, Takeaway, or Missed Shot or it's a goal with no assists
            elif len(player_info) == 2:
                player1_id.append(player_info[0]['player']['id'])
                player1_name.append(player_info[0]['player']['fullName'])
                player1_link.append(player_info[0]['player']['link'])
                player1_type.append(player_info[0]['playerType'])
                player2_id.append(player_info[1]['player']['id'])
                player2_name.append(player_info[1]['player']['fullName'])
                player2_link.append(player_info[1]['player']['link'])
                player2_type.append(player_info[1]['playerType'])   
                primaryAssister_id.append('N/A')
                primaryAssister_name.append('N/A')
                primaryAssister_link.append('N/A')
                primaryAssister_type.append('N/A')
                primaryAssister_total.append('N/A')
                secondaryAssister_id.append('N/A')
                secondaryAssister_name.append('N/A')
                secondaryAssister_link.append('N/A')
                secondaryAssister_type.append('N/A')
                secondaryAssister_total.append('N/A')
                penaltyServer_id.append('N/A')
                penaltyServer_name.append('N/A')
                penaltyServer_link.append('N/A')
                penaltyServer_type.append('N/A')
            # event type is a Goal with one assist or a Penalty taken and served by different players
            elif len(player_info) == 3:
                player1_id.append(player_info[0]['player']['id'])
                player1_name.append(player_info[0]['player']['fullName'])
                player1_link.append(player_info[0]['player']['link'])
                player1_type.append(player_info[0]['playerType'])
                # event type is Goal
                if event_result['eventTypeId'] == 'GOAL':
                    primaryAssister_id.append(player_info[1]['player']['id'])
                    primaryAssister_name.append(player_info[1]['player']['fullName'])
                    primaryAssister_link.append(player_info[1]['player']['link'])
                    primaryAssister_type.append(player_info[1]['playerType'])
                    primaryAssister_total.append(player_info[1]['seasonTotal'])
                    player2_id.append(player_info[2]['player']['id'])
                    player2_name.append(player_info[2]['player']['fullName'])
                    player2_link.append(player_info[2]['player']['link'])
                    player2_type.append(player_info[2]['playerType'])
                    penaltyServer_id.append('N/A')
                    penaltyServer_name.append('N/A')
                    penaltyServer_link.append('N/A')
                    penaltyServer_type.append('N/A')
                # event type is Penalty
                elif event_result['eventTypeId'] == 'PENALTY':
                    player2_id.append(player_info[1]['player']['id'])
                    player2_name.append(player_info[1]['player']['fullName'])
                    player2_link.append(player_info[1]['player']['link'])
                    player2_type.append(player_info[1]['playerType'])
                    penaltyServer_id.append(player_info[2]['player']['id'])
                    penaltyServer_name.append(player_info[2]['player']['fullName'])
                    penaltyServer_link.append(player_info[2]['player']['link'])
                    penaltyServer_type.append(player_info[2]['playerType'])
                    primaryAssister_id.append('N/A')
                    primaryAssister_name.append('N/A')
                    primaryAssister_link.append('N/A')
                    primaryAssister_type.append('N/A')
                    primaryAssister_total.append('N/A')
                # who fucking knows
                else:
                    print(game_data['game']['pk'])   
                secondaryAssister_id.append('N/A')
                secondaryAssister_name.append('N/A')
                secondaryAssister_link.append('N/A')
                secondaryAssister_type.append('N/A')
                secondaryAssister_total.append('N/A')
            # event type is a Goal with two assists
            elif len(player_info) == 4:
                player1_id.append(player_info[0]['player']['id'])
                player1_name.append(player_info[0]['player']['fullName'])
                player1_link.append(player_info[0]['player']['link'])
                player1_type.append(player_info[0]['playerType'])
                player2_id.append(player_info[3]['player']['id'])
                player2_name.append(player_info[3]['player']['fullName'])
                player2_link.append(player_info[3]['player']['link'])
                player2_type.append(player_info[3]['playerType']) 
                primaryAssister_id.append(player_info[1]['player']['id'])
                primaryAssister_name.append(player_info[1]['player']['fullName'])
                primaryAssister_link.append(player_info[1]['player']['link'])
                primaryAssister_type.append(player_info[1]['playerType'])
                primaryAssister_total.append(player_info[1]['seasonTotal'])
                secondaryAssister_id.append(player_info[2]['player']['id'])
                secondaryAssister_name.append(player_info[2]['player']['fullName'])
                secondaryAssister_link.append(player_info[2]['player']['link'])
                secondaryAssister_type.append(player_info[2]['playerType'])
                secondaryAssister_total.append(player_info[2]['seasonTotal'])
                penaltyServer_id.append('N/A')
                penaltyServer_name.append('N/A')
                penaltyServer_link.append('N/A')
                penaltyServer_type.append('N/A')
            else:
                print('What the fuck')

            # event is a Goal, Shot, or Penalty
            if 'secondaryType' in event_result:
                secondaryType.append(event_result['secondaryType'])
            else:
                secondaryType.append('N/A')

            # event is a Penalty
            if event_result['eventTypeId'] == 'PENALTY':
                penaltySeverity.append(event_result['penaltySeverity'])
                penaltyMinutes.append(event_result['penaltyMinutes'])
            else:
                penaltySeverity.append('N/A')
                penaltyMinutes.append('N/A')

            # event is a Goal
            if event_result['eventTypeId'] == 'GOAL':
                strength.append(event_result['strength']['code'])
                gwg.append(event_result['gameWinningGoal'])
                # not sure why but some goal events are missing emptyNet key
                if 'emptyNet' in event_result:
                    emptyNet.append(event_result['emptyNet'])
                else:
                    emptyNet.append('N/A')
                scorer_total.append(player_info[0]['seasonTotal'])
            else:
                strength.append('N/A')
                gwg.append('N/A')
                emptyNet.append('N/A')
                scorer_total.append('N/A')


            # regardless of event type
            eventIdx.append(event_about['eventIdx'])
            event_id.append(str(game_data['game']['pk']) + '-' + str(event_about['eventIdx'])) # primary key for nhl_events
            event.append(event_result['event'])
            eventCode.append(event_result['eventCode'])
            eventType.append(event_result['eventTypeId'])
            eventDescription.append(event_result['description'])
            period.append(event_about['period'])
            periodType.append(event_about['periodType'])
            periodTime.append(event_about['periodTime'])
            dateTime.append(event_about['dateTime'])
            awayGoals.append(event_about['goals']['away'])
            homeGoals.append(event_about['goals']['home'])
            # some events are missing coordinates, no idea why
            if events[i]['coordinates']:
                xCoord.append(events[i]['coordinates']['x'])
                yCoord.append(events[i]['coordinates']['y'])
            else:
                xCoord.append('Missing')
                yCoord.append('Missing')
            team1_id.append(events[i]['team']['id'])
            team1_name.append(events[i]['team']['name'])
            team1_link.append(events[i]['team']['link'])
            team1_tricode.append(events[i]['team']['triCode'])

            # players on ice for each event
            away_players_on_ice = []
            home_players_on_ice = []
            for shift in range(len(shift_df)):
                if event_during_shift(event_about['periodTime'], event_about['period'], shift_df['shift start'][shift],
                                        shift_df['shift end'][shift], shift_df['period'][shift]) == True:
#                     print(1)
                    if shift_df['player team'][shift] == away_abbrev:
                        away_players_on_ice.append(shift_df['player name'][shift])
                    elif shift_df['player team'][shift] == home_abbrev:
                        home_players_on_ice.append(shift_df['player name'][shift])

            if len(away_players_on_ice)>=1:
                away_p1.append(away_players_on_ice[0])
            else:
                away_p1.append('N/A')
            if len(away_players_on_ice)>=2:
                away_p2.append(away_players_on_ice[1])
            else:
                away_p2.append('N/A')
            if len(away_players_on_ice)>=3:
                away_p3.append(away_players_on_ice[2])
            else:
                away_p3.append('N/A')
            if len(away_players_on_ice)>=4:
                away_p4.append(away_players_on_ice[3])
            else:
                away_p4.append('N/A')
            if len(away_players_on_ice)>=5:
                away_p5.append(away_players_on_ice[4])
            else:
                away_p5.append('N/A')
            if len(away_players_on_ice)>=6:
                away_p6.append(away_players_on_ice[5])
            else:
                away_p6.append('N/A')

            if len(home_players_on_ice)>=1:
                home_p1.append(home_players_on_ice[0])
            else:
                home_p1.append('N/A')
            if len(home_players_on_ice)>=2:
                home_p2.append(home_players_on_ice[1])
            else:
                home_p2.append('N/A')
            if len(home_players_on_ice)>=3:
                home_p3.append(home_players_on_ice[2])
            else:
                home_p3.append('N/A')
            if len(home_players_on_ice)>=4:
                home_p4.append(home_players_on_ice[3])
            else:
                home_p4.append('N/A')
            if len(home_players_on_ice)>=5:
                home_p5.append(home_players_on_ice[4])
            else:
                home_p5.append('N/A')
            if len(home_players_on_ice)>=6:
                home_p6.append(home_players_on_ice[5])
            else:
                home_p6.append('N/A')


        # event is a non-hockey event (Game start/end, Period start/end, Stoppage, etc.)
        else:
            player1_id.append('N/A')
            player1_name.append('N/A')
            player1_link.append('N/A')
            player1_type.append('N/A')
            player2_id.append('N/A')
            player2_name.append('N/A')
            player2_link.append('N/A')
            player2_type.append('N/A')
            primaryAssister_id.append('N/A')
            primaryAssister_name.append('N/A')
            primaryAssister_link.append('N/A')
            primaryAssister_type.append('N/A')
            primaryAssister_total.append('N/A')
            secondaryAssister_id.append('N/A')
            secondaryAssister_name.append('N/A')
            secondaryAssister_link.append('N/A')
            secondaryAssister_type.append('N/A')
            secondaryAssister_total.append('N/A')
            penaltyServer_id.append('N/A')
            penaltyServer_name.append('N/A')
            penaltyServer_link.append('N/A')
            penaltyServer_type.append('N/A')
            strength.append('N/A')
            gwg.append('N/A')
            emptyNet.append('N/A')
            scorer_total.append('N/A')
            event.append(event_result['event'])
            eventCode.append(event_result['eventCode'])
            eventType.append(event_result['eventTypeId'])
            eventDescription.append(event_result['description'])
            secondaryType.append('N/A')
            penaltySeverity.append('N/A')
            penaltyMinutes.append('N/A')
            eventIdx.append(event_about['eventIdx'])
            event_id.append(str(game_data['game']['pk']) + '-' + str(event_about['eventIdx'])) # primary key for nhl_events
            period.append(event_about['period'])
            periodType.append(event_about['periodType'])
            periodTime.append(event_about['periodTime'])
            dateTime.append(event_about['dateTime'])
            awayGoals.append(event_about['goals']['away'])
            homeGoals.append(event_about['goals']['home'])
            xCoord.append('N/A')
            yCoord.append('N/A')
            team1_id.append('N/A')
            team1_name.append('N/A')
            team1_link.append('N/A')
            team1_tricode.append('N/A')
            away_p1.append('N/A')
            away_p2.append('N/A')
            away_p3.append('N/A')
            away_p4.append('N/A')
            away_p5.append('N/A')
            away_p6.append('N/A')
            home_p1.append('N/A')
            home_p2.append('N/A')
            home_p3.append('N/A')
            home_p4.append('N/A')
            home_p5.append('N/A')
            home_p6.append('N/A')

# convert resulting season-long lists into Dataframe
reg20192020_events_df = pd.DataFrame({
    'event_id': event_id,
    'game_id': game_id,
    'player1_id': player1_id,
    'player1_name': player1_name,
    'player1_link': player1_link,
    'player1_type': player1_type,
    'player2_id': player2_id,
    'player2_name': player2_name,
    'player2_link': player2_link,
    'player2_type': player2_type,
    'event': event,
    'eventCode': eventCode,
    'eventType': eventType,
    'eventDescription': eventDescription,
    'secondaryType': secondaryType,
    'primaryAssister_id': primaryAssister_id,
    'primaryAssister_name': primaryAssister_name,
    'primaryAssister_link': primaryAssister_link,
    'primaryAssister_type': primaryAssister_type,
    'primaryAssister_total': primaryAssister_total,
    'secondaryAssister_id': secondaryAssister_id,
    'secondaryAssister_name': secondaryAssister_name,
    'secondaryAssister_link': secondaryAssister_link,
    'secondaryAssister_type': secondaryAssister_type,
    'secondaryAssister_total': secondaryAssister_total,
    'penaltyServer_id': penaltyServer_id,
    'penaltyServer_name': penaltyServer_name,
    'penaltyServer_link': penaltyServer_link,
    'penaltyServer_type': penaltyServer_type,
    'strength': strength,
    'gwg': gwg,
    'emptyNet': emptyNet,
    'scorer_total': scorer_total,
    'penaltySeverity': penaltySeverity,
    'penaltyMinutes': penaltyMinutes,
    'eventIdx': eventIdx,
    'period': period,
    'periodType': periodType,
    'periodTime': periodTime,
    'dateTime': dateTime,
    'awayGoals': awayGoals,
    'homeGoals': homeGoals,
    'xCoord': xCoord,
    'yCoord': yCoord,
    'team1_id': team1_id,
    'team1_name': team1_name,
    'team1_link': team1_link,
    'team1_tricode': team1_tricode,
    'away p1': away_p1,
    'away p2': away_p2,
    'away p3': away_p3,
    'away p4': away_p4,
    'away p5': away_p5,
    'away p6': away_p6,
    'home p1': home_p1,
    'home p2': home_p2,
    'home p3': home_p3,
    'home p4': home_p4,
    'home p5': home_p5,
    'home p6': home_p6
})

# reg20192020_games_df = {
#     'game_id': game_id[0],
#     'game_link': game_link,
#     'season': season,
#     'game_type': game_type,
#     'game_status': game_status,      
#     'away_team_id': away_team_id,
#     'away_team_link': away_team_link,
#     'away_team_name': away_team_name,
#     'away_team_abbrev': away_team_abbrev,
#     'away_team_result': away_team_result,
#     'away_team_goals': away_team_goals,
#     'away_coach': away_coach,
#     'home_team_id': home_team_id,
#     'home_team_link': home_team_link,
#     'home_team_name': home_team_name,
#     'home_team_abbrev': home_team_abbrev,
#     'home_team_result': home_team_result, # if home_team_goals < away_team goals
#     'home_team_goals': home_team_goals,
#     'home_coach': home_coach,
#     'game_length': game_length,
#     'game_start_time': game_start_time,
#     'game_end_time': game_end_time,
#     'referee1': referee1,
#     'referee2': referee2,
#     'linesman1': linesman1,
#     'linesman2': linesman2
# }

In [58]:
reg20192020_events_df[2740:2800]

,event_id,game_id,player1_id,player1_name,player1_link,player1_type,player2_id,player2_name,player2_link,player2_type,event,eventCode,eventType,eventDescription,secondaryType,primaryAssister_id,primaryAssister_name,primaryAssister_link,primaryAssister_type,primaryAssister_total,secondaryAssister_id,secondaryAssister_name,secondaryAssister_link,secondaryAssister_type,secondaryAssister_total,penaltyServer_id,penaltyServer_name,penaltyServer_link,penaltyServer_type,strength,gwg,emptyNet,scorer_total,penaltySeverity,penaltyMinutes,eventIdx,period,periodType,periodTime,dateTime,awayGoals,homeGoals,xCoord,yCoord,team1_id,team1_name,team1_link,team1_tricode,away p1,away p2,away p3,away p4,away p5,away p6,home p1,home p2,home p3,home p4,home p5,home p6
2740,2019020008-360,2019020008,8474038,Paul Byron,/api/v1/people/8474038,Shooter,N/A,N/A,N/A,N/A,Missed Shot,CAR690,MISSED_SHOT,Paul Byron Wide of Net,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,360,4,OVERTIME,00:27,2019-10-04T01:46:06Z,3,3,-75.0,4.0,8,Montréal Canadiens,/api/v1/teams/8,MTL,Carey Price,Jeff Petry,Paul Byron,Max Domi,N/A,N/A,Petr Mrazek,Jaccob Slavin,Sebastian Aho,Andrei Svechnikov,N/A,N/A
2741,2019020008-361,2019020008,8474038,Paul Byron,/api/v1/people/8474038,Blocker,8476462,Dougie Hamilton,/api/v1/people/8476462,Shooter,Blocked Shot,CAR691,BLOCKED_SHOT,Dougie Hamilton shot blocked shot by Paul Byron,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,361,4,OVERTIME,00:44,2019-10-04T01:46:22Z,3,3,69.0,-8.0,8,Montréal Canadiens,/api/v1/teams/8,MTL,Carey Price,Jeff Petry,Paul Byron,Max Domi,N/A,N/A,Petr Mrazek,Dougie Hamilton,Teuvo Teravainen,Sebastian Aho,N/A,N/A
2742,2019020008-362,2019020008,8474038,Paul Byron,/api/v1/people/8474038,Blocker,8476462,Dougie Hamilton,/api/v1/people/8476462,Shooter,Blocked Shot,CAR692,BLOCKED_SHOT,Dougie Hamilton shot blocked shot by Paul Byron,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,362,4,OVERTIME,00:53,2019-10-04T01:46:31Z,3,3,65.0,-5.0,8,Montréal Canadiens,/api/v1/teams/8,MTL,Carey Price,Jeff Petry,Paul Byron,Max Domi,N/A,N/A,Petr Mrazek,Dougie Hamilton,Teuvo Teravainen,Sebastian Aho,N/A,N/A
2743,2019020008-363,2019020008,8474581,Jake Gardiner,/api/v1/people/8474581,Shooter,8471679,Carey Price,/api/v1/people/8471679,Goalie,Shot,CAR693,SHOT,Jake Gardiner Slap Shot saved by Carey Price,Slap Shot,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,363,4,OVERTIME,01:42,2019-10-04T01:47:20Z,3,3,46.0,-27.0,12,Carolina Hurricanes,/api/v1/teams/12,CAR,Carey Price,Brendan Gallagher,Jonathan Drouin,Victor Mete,N/A,N/A,Jake Gardiner,Erik Haula,Petr Mrazek,Teuvo Teravainen,N/A,N/A
2744,2019020008-364,2019020008,8479376,Victor Mete,/api/v1/people/8479376,Hitter,8475287,Erik Haula,/api/v1/people/8475287,Hittee,Hit,CAR648,HIT,Victor Mete hit Erik Haula,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,364,4,OVERTIME,02:09,2019-10-04T01:47:49Z,3,3,70.0,40.0,8,Montréal Canadiens,/api/v1/teams/8,MTL,Carey Price,Brendan Gallagher,Jonathan Drouin,Victor Mete,N/A,N/A,Erik Haula,Petr Mrazek,Jaccob Slavin,Martin Necas,N/A,N/A
2745,2019020008-365,2019020008,8480829,Jesperi Kotkaniemi,/api/v1/people/8480829,Shooter,8475852,Petr Mrazek,/api/v1/people/8475852,Goalie,Shot,CAR694,SHOT,Jesperi Kotkaniemi Wrist Shot saved by Petr Mr...,Wrist Shot,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,365,4,OVERTIME,03:02,2019-10-04T01:48:40Z,3,3,-66.0,1.0,8,Montréal Canadiens,/api/v1/teams/8,MTL,Shea Weber,Carey Price,Joel Armia,Jesperi Kotkaniemi,N/A,N/A,Jordan Staal,Petr Mrazek,Jaccob Slavin,Andrei Svechnikov,N/A,N/A
2746,2019020008-366,2019020008,8476958,Jaccob Slavin,/api/v1/people/8476958,Hitter,8477503,Max Domi,/api/v1/people/8477503,Hittee,Hit,CAR649,HIT,Jaccob Slavin hit Max Domi,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,

# 2020-2021 Regular Season

In [ ]:
# create lists to store events for the entire season, to be converted to Dataframe
event_id = []
game_id = []
player1_id = []
player1_name = []
player1_link = []
player1_type = []
player2_id = []
player2_name = []
player2_link = []
player2_type = []
primaryAssister_id = []
primaryAssister_name = []
primaryAssister_link = []
primaryAssister_type = []
primaryAssister_total = []
secondaryAssister_id = []
secondaryAssister_name = []
secondaryAssister_link = []
secondaryAssister_type = []
secondaryAssister_total = []
penaltyServer_id = []
penaltyServer_name = []
penaltyServer_link = []
penaltyServer_type = []
strength = []
gwg = []
emptyNet = []
scorer_total = []
event = []
eventCode = []
eventType = []
eventDescription = []
secondaryType = []
penaltySeverity = []
penaltyMinutes = []
eventIdx = []
period = []
periodType = []
periodTime = []
dateTime = []
awayGoals = []
homeGoals = []
xCoord = []
yCoord = []
team1_id = []
team1_name = []
team1_link = []
team1_tricode = []

# 2020-2021 reg. season only had 56 games per team and only 868 games total
root = '202002'
for game in range(1, 869):
    if game<10:
        code = root+'000'+str(game)
    elif game<100:
        code = root+'00'+str(game)
    elif game<1000:
        code = root+'0'+str(game)
    else:
        code = root+str(game)
        
            
    url = 'https://statsapi.web.nhl.com/api/v1/game/'+str(code)+'/feed/live'
    
    # access page and pull relevant JSON
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    page_json = json.loads(webpage)
    game_data = page_json['gameData']
    events = page_json['liveData']['plays']['allPlays']
    
#     print(game_data['game']['pk'])
    
    # iterate through all events and store as a row
    for i in range(len(events)):
    
        # add game_id to every row
        game_id.append(game_data['game']['pk'])

        # these keys exist for every event, whether hockey play or not
        event_result = events[i]['result']
        event_about = events[i]['about']
    
        # event is a hockey event (Shot, Goal, Hit, etc.)
        if 'players' in events[i]:

            player_info = events[i]['players']

            # event type is a Giveaway, Takeaway, or Missed Shot
            if len(player_info) == 1:
                player1_id.append(player_info[0]['player']['id'])
                player1_name.append(player_info[0]['player']['fullName'])
                player1_link.append(player_info[0]['player']['link'])
                player1_type.append(player_info[0]['playerType'])
                player2_id.append('N/A')
                player2_name.append('N/A')
                player2_link.append('N/A')
                player2_type.append('N/A')
                primaryAssister_id.append('N/A')
                primaryAssister_name.append('N/A')
                primaryAssister_link.append('N/A')
                primaryAssister_type.append('N/A')
                primaryAssister_total.append('N/A')
                secondaryAssister_id.append('N/A')
                secondaryAssister_name.append('N/A')
                secondaryAssister_link.append('N/A')
                secondaryAssister_type.append('N/A')
                secondaryAssister_total.append('N/A')
                penaltyServer_id.append('N/A')
                penaltyServer_name.append('N/A')
                penaltyServer_link.append('N/A')
                penaltyServer_type.append('N/A')
            # event type is not a Goal, Giveaway, Takeaway, or Missed Shot or it's a goal with no assists
            elif len(player_info) == 2:
                player1_id.append(player_info[0]['player']['id'])
                player1_name.append(player_info[0]['player']['fullName'])
                player1_link.append(player_info[0]['player']['link'])
                player1_type.append(player_info[0]['playerType'])
                player2_id.append(player_info[1]['player']['id'])
                player2_name.append(player_info[1]['player']['fullName'])
                player2_link.append(player_info[1]['player']['link'])
                player2_type.append(player_info[1]['playerType'])   
                primaryAssister_id.append('N/A')
                primaryAssister_name.append('N/A')
                primaryAssister_link.append('N/A')
                primaryAssister_type.append('N/A')
                primaryAssister_total.append('N/A')
                secondaryAssister_id.append('N/A')
                secondaryAssister_name.append('N/A')
                secondaryAssister_link.append('N/A')
                secondaryAssister_type.append('N/A')
                secondaryAssister_total.append('N/A')
                penaltyServer_id.append('N/A')
                penaltyServer_name.append('N/A')
                penaltyServer_link.append('N/A')
                penaltyServer_type.append('N/A')
            # event type is a Goal with one assist or a Penalty taken and served by different players
            elif len(player_info) == 3:
                player1_id.append(player_info[0]['player']['id'])
                player1_name.append(player_info[0]['player']['fullName'])
                player1_link.append(player_info[0]['player']['link'])
                player1_type.append(player_info[0]['playerType'])
                # event type is Goal
                if event_result['eventTypeId'] == 'GOAL':
                    primaryAssister_id.append(player_info[1]['player']['id'])
                    primaryAssister_name.append(player_info[1]['player']['fullName'])
                    primaryAssister_link.append(player_info[1]['player']['link'])
                    primaryAssister_type.append(player_info[1]['playerType'])
                    primaryAssister_total.append(player_info[1]['seasonTotal'])
                    player2_id.append(player_info[2]['player']['id'])
                    player2_name.append(player_info[2]['player']['fullName'])
                    player2_link.append(player_info[2]['player']['link'])
                    player2_type.append(player_info[2]['playerType'])
                    penaltyServer_id.append('N/A')
                    penaltyServer_name.append('N/A')
                    penaltyServer_link.append('N/A')
                    penaltyServer_type.append('N/A')
                # event type is Penalty
                elif event_result['eventTypeId'] == 'PENALTY':
                    player2_id.append(player_info[1]['player']['id'])
                    player2_name.append(player_info[1]['player']['fullName'])
                    player2_link.append(player_info[1]['player']['link'])
                    player2_type.append(player_info[1]['playerType'])
                    penaltyServer_id.append(player_info[2]['player']['id'])
                    penaltyServer_name.append(player_info[2]['player']['fullName'])
                    penaltyServer_link.append(player_info[2]['player']['link'])
                    penaltyServer_type.append(player_info[2]['playerType'])
                    primaryAssister_id.append('N/A')
                    primaryAssister_name.append('N/A')
                    primaryAssister_link.append('N/A')
                    primaryAssister_type.append('N/A')
                    primaryAssister_total.append('N/A')
                # who fucking knows
                else:
                    print(game_data['game']['pk'])   
                secondaryAssister_id.append('N/A')
                secondaryAssister_name.append('N/A')
                secondaryAssister_link.append('N/A')
                secondaryAssister_type.append('N/A')
                secondaryAssister_total.append('N/A')
            # event type is a Goal with two assists
            elif len(player_info) == 4:
                player1_id.append(player_info[0]['player']['id'])
                player1_name.append(player_info[0]['player']['fullName'])
                player1_link.append(player_info[0]['player']['link'])
                player1_type.append(player_info[0]['playerType'])
                player2_id.append(player_info[3]['player']['id'])
                player2_name.append(player_info[3]['player']['fullName'])
                player2_link.append(player_info[3]['player']['link'])
                player2_type.append(player_info[3]['playerType']) 
                primaryAssister_id.append(player_info[1]['player']['id'])
                primaryAssister_name.append(player_info[1]['player']['fullName'])
                primaryAssister_link.append(player_info[1]['player']['link'])
                primaryAssister_type.append(player_info[1]['playerType'])
                primaryAssister_total.append(player_info[1]['seasonTotal'])
                secondaryAssister_id.append(player_info[2]['player']['id'])
                secondaryAssister_name.append(player_info[2]['player']['fullName'])
                secondaryAssister_link.append(player_info[2]['player']['link'])
                secondaryAssister_type.append(player_info[2]['playerType'])
                secondaryAssister_total.append(player_info[2]['seasonTotal'])
                penaltyServer_id.append('N/A')
                penaltyServer_name.append('N/A')
                penaltyServer_link.append('N/A')
                penaltyServer_type.append('N/A')
            else:
                print('What the fuck')



            # event is a Goal, Shot, or Penalty
            if 'secondaryType' in event_result:
                secondaryType.append(event_result['secondaryType'])
            else:
                secondaryType.append('N/A')



            # event is a Penalty
            if event_result['eventTypeId'] == 'PENALTY':
                penaltySeverity.append(event_result['penaltySeverity'])
                penaltyMinutes.append(event_result['penaltyMinutes'])
            else:
                penaltySeverity.append('N/A')
                penaltyMinutes.append('N/A')



            # event is a Goal
            if event_result['eventTypeId'] == 'GOAL':
                strength.append(event_result['strength']['code'])
                gwg.append(event_result['gameWinningGoal'])
                # not sure why but some goal events are missing emptyNet key
                if 'emptyNet' in event_result:
                    emptyNet.append(event_result['emptyNet'])
                else:
                    emptyNet.append('N/A')
                scorer_total.append(player_info[0]['seasonTotal'])
            else:
                strength.append('N/A')
                gwg.append('N/A')
                emptyNet.append('N/A')
                scorer_total.append('N/A')



            eventIdx.append(event_about['eventIdx'])
            event_id.append(str(game_data['game']['pk']) + '-' + str(event_about['eventIdx'])) # primary key for nhl_events
            event.append(event_result['event'])
            eventCode.append(event_result['eventCode'])
            eventType.append(event_result['eventTypeId'])
            eventDescription.append(event_result['description'])
            period.append(event_about['period'])
            periodType.append(event_about['periodType'])
            periodTime.append(event_about['periodTime'])
            dateTime.append(event_about['dateTime'])
            awayGoals.append(event_about['goals']['away'])
            homeGoals.append(event_about['goals']['home'])
            # some events are missing coordinates, no idea why
            if events[i]['coordinates']:
                xCoord.append(events[i]['coordinates']['x'])
                yCoord.append(events[i]['coordinates']['y'])
            else:
                xCoord.append('Missing')
                yCoord.append('Missing')
            team1_id.append(events[i]['team']['id'])
            team1_name.append(events[i]['team']['name'])
            team1_link.append(events[i]['team']['link'])
            team1_tricode.append(events[i]['team']['triCode'])



        # event is a non-hockey event (Game start/end, Period start/end, Stoppage, etc.)
        else:
            player1_id.append('N/A')
            player1_name.append('N/A')
            player1_link.append('N/A')
            player1_type.append('N/A')
            player2_id.append('N/A')
            player2_name.append('N/A')
            player2_link.append('N/A')
            player2_type.append('N/A')
            primaryAssister_id.append('N/A')
            primaryAssister_name.append('N/A')
            primaryAssister_link.append('N/A')
            primaryAssister_type.append('N/A')
            primaryAssister_total.append('N/A')
            secondaryAssister_id.append('N/A')
            secondaryAssister_name.append('N/A')
            secondaryAssister_link.append('N/A')
            secondaryAssister_type.append('N/A')
            secondaryAssister_total.append('N/A')
            penaltyServer_id.append('N/A')
            penaltyServer_name.append('N/A')
            penaltyServer_link.append('N/A')
            penaltyServer_type.append('N/A')
            strength.append('N/A')
            gwg.append('N/A')
            emptyNet.append('N/A')
            scorer_total.append('N/A')
            event.append(event_result['event'])
            eventCode.append(event_result['eventCode'])
            eventType.append(event_result['eventTypeId'])
            eventDescription.append(event_result['description'])
            secondaryType.append('N/A')
            penaltySeverity.append('N/A')
            penaltyMinutes.append('N/A')
            eventIdx.append(event_about['eventIdx'])
            event_id.append(str(game_data['game']['pk']) + '-' + str(event_about['eventIdx'])) # primary key for nhl_events
            period.append(event_about['period'])
            periodType.append(event_about['periodType'])
            periodTime.append(event_about['periodTime'])
            dateTime.append(event_about['dateTime'])
            awayGoals.append(event_about['goals']['away'])
            homeGoals.append(event_about['goals']['home'])
            xCoord.append('N/A')
            yCoord.append('N/A')
            team1_id.append('N/A')
            team1_name.append('N/A')
            team1_link.append('N/A')
            team1_tricode.append('N/A')

# convert resulting season-long lists into Dataframe
reg20202021_events_df = pd.DataFrame({
    'event_id': event_id,
    'game_id': game_id,
    'player1_id': player1_id,
    'player1_name': player1_name,
    'player1_link': player1_link,
    'player1_type': player1_type,
    'player2_id': player2_id,
    'player2_name': player2_name,
    'player2_link': player2_link,
    'player2_type': player2_type,
    'event': event,
    'eventCode': eventCode,
    'eventType': eventType,
    'eventDescription': eventDescription,
    'secondaryType': secondaryType,
    'primaryAssister_id': primaryAssister_id,
    'primaryAssister_name': primaryAssister_name,
    'primaryAssister_link': primaryAssister_link,
    'primaryAssister_type': primaryAssister_type,
    'primaryAssister_total': primaryAssister_total,
    'secondaryAssister_id': secondaryAssister_id,
    'secondaryAssister_name': secondaryAssister_name,
    'secondaryAssister_link': secondaryAssister_link,
    'secondaryAssister_type': secondaryAssister_type,
    'secondaryAssister_total': secondaryAssister_total,
    'penaltyServer_id': penaltyServer_id,
    'penaltyServer_name': penaltyServer_name,
    'penaltyServer_link': penaltyServer_link,
    'penaltyServer_type': penaltyServer_type,
    'strength': strength,
    'gwg': gwg,
    'emptyNet': emptyNet,
    'scorer_total': scorer_total,
    'penaltySeverity': penaltySeverity,
    'penaltyMinutes': penaltyMinutes,
    'eventIdx': eventIdx,
    'period': period,
    'periodType': periodType,
    'periodTime': periodTime,
    'dateTime': dateTime,
    'awayGoals': awayGoals,
    'homeGoals': homeGoals,
    'xCoord': xCoord,
    'yCoord': yCoord,
    'team1_id': team1_id,
    'team1_name': team1_name,
    'team1_link': team1_link,
    'team1_tricode': team1_tricode
})

In [ ]:
reg20202021_events_df[5755:5760]

# 2021-2022 Regular Season

## work

In [ ]:
# create lists to store events for the entire season, to be converted to Dataframe
event_id = []
game_id = []
player1_id = []
player1_name = []
player1_link = []
player1_type = []
player2_id = []
player2_name = []
player2_link = []
player2_type = []
primaryAssister_id = []
primaryAssister_name = []
primaryAssister_link = []
primaryAssister_type = []
primaryAssister_total = []
secondaryAssister_id = []
secondaryAssister_name = []
secondaryAssister_link = []
secondaryAssister_type = []
secondaryAssister_total = []
penaltyServer_id = []
penaltyServer_name = []
penaltyServer_link = []
penaltyServer_type = []
strength = []
gwg = []
emptyNet = []
scorer_total = []
event = []
eventCode = []
eventType = []
eventDescription = []
secondaryType = []
penaltySeverity = []
penaltyMinutes = []
eventIdx = []
period = []
periodType = []
periodTime = []
dateTime = []
awayGoals = []
homeGoals = []
xCoord = []
yCoord = []
team1_id = []
team1_name = []
team1_link = []
team1_tricode = []

# 2020-2021 reg. season only had 56 games per team and only 868 games total
root = '202102'
for game in range(1, 1400):
    if game<10:
        code = root+'000'+str(game)
    elif game<100:
        code = root+'00'+str(game)
    elif game<1000:
        code = root+'0'+str(game)
    else:
        code = root+str(game)
        
            
    url = 'https://statsapi.web.nhl.com/api/v1/game/'+str(code)+'/feed/live'
    
    # access page and pull relevant JSON
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    
    # check if valid url
    response = requests.get(url)
    if response.status_code == 200:
        webpage = urlopen(req).read()
        page_json = json.loads(webpage)
        game_data = page_json['gameData']
        
        # check if game has been played
        if game_data['status']['abstractGameState'] == "Final":
            events = page_json['liveData']['plays']['allPlays']

        #     print(game_data['game']['pk'])

            # iterate through all events and store as a row
            for i in range(len(events)):

                # add game_id to every row
                game_id.append(game_data['game']['pk'])

                # these keys exist for every event, whether hockey play or not
                event_result = events[i]['result']
                event_about = events[i]['about']

                # event is a hockey event (Shot, Goal, Hit, etc.)
                if 'players' in events[i]:

                    player_info = events[i]['players']

                    # event type is a Giveaway, Takeaway, or Missed Shot
                    if len(player_info) == 1:
                        player1_id.append(player_info[0]['player']['id'])
                        player1_name.append(player_info[0]['player']['fullName'])
                        player1_link.append(player_info[0]['player']['link'])
                        player1_type.append(player_info[0]['playerType'])
                        player2_id.append('N/A')
                        player2_name.append('N/A')
                        player2_link.append('N/A')
                        player2_type.append('N/A')
                        primaryAssister_id.append('N/A')
                        primaryAssister_name.append('N/A')
                        primaryAssister_link.append('N/A')
                        primaryAssister_type.append('N/A')
                        primaryAssister_total.append('N/A')
                        secondaryAssister_id.append('N/A')
                        secondaryAssister_name.append('N/A')
                        secondaryAssister_link.append('N/A')
                        secondaryAssister_type.append('N/A')
                        secondaryAssister_total.append('N/A')
                        penaltyServer_id.append('N/A')
                        penaltyServer_name.append('N/A')
                        penaltyServer_link.append('N/A')
                        penaltyServer_type.append('N/A')
                    # event type is not a Goal, Giveaway, Takeaway, or Missed Shot or it's a goal with no assists
                    elif len(player_info) == 2:
                        player1_id.append(player_info[0]['player']['id'])
                        player1_name.append(player_info[0]['player']['fullName'])
                        player1_link.append(player_info[0]['player']['link'])
                        player1_type.append(player_info[0]['playerType'])
                        player2_id.append(player_info[1]['player']['id'])
                        player2_name.append(player_info[1]['player']['fullName'])
                        player2_link.append(player_info[1]['player']['link'])
                        player2_type.append(player_info[1]['playerType'])   
                        primaryAssister_id.append('N/A')
                        primaryAssister_name.append('N/A')
                        primaryAssister_link.append('N/A')
                        primaryAssister_type.append('N/A')
                        primaryAssister_total.append('N/A')
                        secondaryAssister_id.append('N/A')
                        secondaryAssister_name.append('N/A')
                        secondaryAssister_link.append('N/A')
                        secondaryAssister_type.append('N/A')
                        secondaryAssister_total.append('N/A')
                        penaltyServer_id.append('N/A')
                        penaltyServer_name.append('N/A')
                        penaltyServer_link.append('N/A')
                        penaltyServer_type.append('N/A')
                    # event type is a Goal with one assist or a Penalty taken and served by different players
                    elif len(player_info) == 3:
                        player1_id.append(player_info[0]['player']['id'])
                        player1_name.append(player_info[0]['player']['fullName'])
                        player1_link.append(player_info[0]['player']['link'])
                        player1_type.append(player_info[0]['playerType'])
                        # event type is Goal
                        if event_result['eventTypeId'] == 'GOAL':
                            primaryAssister_id.append(player_info[1]['player']['id'])
                            primaryAssister_name.append(player_info[1]['player']['fullName'])
                            primaryAssister_link.append(player_info[1]['player']['link'])
                            primaryAssister_type.append(player_info[1]['playerType'])
                            primaryAssister_total.append(player_info[1]['seasonTotal'])
                            player2_id.append(player_info[2]['player']['id'])
                            player2_name.append(player_info[2]['player']['fullName'])
                            player2_link.append(player_info[2]['player']['link'])
                            player2_type.append(player_info[2]['playerType'])
                            penaltyServer_id.append('N/A')
                            penaltyServer_name.append('N/A')
                            penaltyServer_link.append('N/A')
                            penaltyServer_type.append('N/A')
                        # event type is Penalty
                        elif event_result['eventTypeId'] == 'PENALTY':
                            player2_id.append(player_info[1]['player']['id'])
                            player2_name.append(player_info[1]['player']['fullName'])
                            player2_link.append(player_info[1]['player']['link'])
                            player2_type.append(player_info[1]['playerType'])
                            penaltyServer_id.append(player_info[2]['player']['id'])
                            penaltyServer_name.append(player_info[2]['player']['fullName'])
                            penaltyServer_link.append(player_info[2]['player']['link'])
                            penaltyServer_type.append(player_info[2]['playerType'])
                            primaryAssister_id.append('N/A')
                            primaryAssister_name.append('N/A')
                            primaryAssister_link.append('N/A')
                            primaryAssister_type.append('N/A')
                            primaryAssister_total.append('N/A')
                        # who fucking knows
                        else:
                            print(game_data['game']['pk'])   
                        secondaryAssister_id.append('N/A')
                        secondaryAssister_name.append('N/A')
                        secondaryAssister_link.append('N/A')
                        secondaryAssister_type.append('N/A')
                        secondaryAssister_total.append('N/A')
                    # event type is a Goal with two assists
                    elif len(player_info) == 4:
                        player1_id.append(player_info[0]['player']['id'])
                        player1_name.append(player_info[0]['player']['fullName'])
                        player1_link.append(player_info[0]['player']['link'])
                        player1_type.append(player_info[0]['playerType'])
                        player2_id.append(player_info[3]['player']['id'])
                        player2_name.append(player_info[3]['player']['fullName'])
                        player2_link.append(player_info[3]['player']['link'])
                        player2_type.append(player_info[3]['playerType']) 
                        primaryAssister_id.append(player_info[1]['player']['id'])
                        primaryAssister_name.append(player_info[1]['player']['fullName'])
                        primaryAssister_link.append(player_info[1]['player']['link'])
                        primaryAssister_type.append(player_info[1]['playerType'])
                        primaryAssister_total.append(player_info[1]['seasonTotal'])
                        secondaryAssister_id.append(player_info[2]['player']['id'])
                        secondaryAssister_name.append(player_info[2]['player']['fullName'])
                        secondaryAssister_link.append(player_info[2]['player']['link'])
                        secondaryAssister_type.append(player_info[2]['playerType'])
                        secondaryAssister_total.append(player_info[2]['seasonTotal'])
                        penaltyServer_id.append('N/A')
                        penaltyServer_name.append('N/A')
                        penaltyServer_link.append('N/A')
                        penaltyServer_type.append('N/A')
                    else:
                        print('What the fuck')



                    # event is a Goal, Shot, or Penalty
                    if 'secondaryType' in event_result:
                        secondaryType.append(event_result['secondaryType'])
                    else:
                        secondaryType.append('N/A')



                    # event is a Penalty
                    if event_result['eventTypeId'] == 'PENALTY':
                        penaltySeverity.append(event_result['penaltySeverity'])
                        penaltyMinutes.append(event_result['penaltyMinutes'])
                    else:
                        penaltySeverity.append('N/A')
                        penaltyMinutes.append('N/A')



                    # event is a Goal
                    if event_result['eventTypeId'] == 'GOAL':
                        strength.append(event_result['strength']['code'])
                        gwg.append(event_result['gameWinningGoal'])
                        # not sure why but some goal events are missing emptyNet key
                        if 'emptyNet' in event_result:
                            emptyNet.append(event_result['emptyNet'])
                        else:
                            emptyNet.append('N/A')
                        scorer_total.append(player_info[0]['seasonTotal'])
                    else:
                        strength.append('N/A')
                        gwg.append('N/A')
                        emptyNet.append('N/A')
                        scorer_total.append('N/A')



                    eventIdx.append(event_about['eventIdx'])
                    event_id.append(str(game_data['game']['pk']) + '-' + str(event_about['eventIdx'])) # primary key for nhl_events
                    event.append(event_result['event'])
                    eventCode.append(event_result['eventCode'])
                    eventType.append(event_result['eventTypeId'])
                    eventDescription.append(event_result['description'])
                    period.append(event_about['period'])
                    periodType.append(event_about['periodType'])
                    periodTime.append(event_about['periodTime'])
                    dateTime.append(event_about['dateTime'])
                    awayGoals.append(event_about['goals']['away'])
                    homeGoals.append(event_about['goals']['home'])
                    # some events are missing coordinates, no idea why
                    if events[i]['coordinates']:
                        xCoord.append(events[i]['coordinates']['x'])
                        yCoord.append(events[i]['coordinates']['y'])
                    else:
                        xCoord.append('Missing')
                        yCoord.append('Missing')
                    team1_id.append(events[i]['team']['id'])
                    team1_name.append(events[i]['team']['name'])
                    team1_link.append(events[i]['team']['link'])
                    team1_tricode.append(events[i]['team']['triCode'])



                # event is a non-hockey event (Game start/end, Period start/end, Stoppage, etc.)
                else:
                    player1_id.append('N/A')
                    player1_name.append('N/A')
                    player1_link.append('N/A')
                    player1_type.append('N/A')
                    player2_id.append('N/A')
                    player2_name.append('N/A')
                    player2_link.append('N/A')
                    player2_type.append('N/A')
                    primaryAssister_id.append('N/A')
                    primaryAssister_name.append('N/A')
                    primaryAssister_link.append('N/A')
                    primaryAssister_type.append('N/A')
                    primaryAssister_total.append('N/A')
                    secondaryAssister_id.append('N/A')
                    secondaryAssister_name.append('N/A')
                    secondaryAssister_link.append('N/A')
                    secondaryAssister_type.append('N/A')
                    secondaryAssister_total.append('N/A')
                    penaltyServer_id.append('N/A')
                    penaltyServer_name.append('N/A')
                    penaltyServer_link.append('N/A')
                    penaltyServer_type.append('N/A')
                    strength.append('N/A')
                    gwg.append('N/A')
                    emptyNet.append('N/A')
                    scorer_total.append('N/A')
                    event.append(event_result['event'])
                    eventCode.append(event_result['eventCode'])
                    eventType.append(event_result['eventTypeId'])
                    eventDescription.append(event_result['description'])
                    secondaryType.append('N/A')
                    penaltySeverity.append('N/A')
                    penaltyMinutes.append('N/A')
                    eventIdx.append(event_about['eventIdx'])
                    event_id.append(str(game_data['game']['pk']) + '-' + str(event_about['eventIdx'])) # primary key for nhl_events
                    period.append(event_about['period'])
                    periodType.append(event_about['periodType'])
                    periodTime.append(event_about['periodTime'])
                    dateTime.append(event_about['dateTime'])
                    awayGoals.append(event_about['goals']['away'])
                    homeGoals.append(event_about['goals']['home'])
                    xCoord.append('N/A')
                    yCoord.append('N/A')
                    team1_id.append('N/A')
                    team1_name.append('N/A')
                    team1_link.append('N/A')
                    team1_tricode.append('N/A')
                    
        else:
            print('Game has not been played: ', url)

    else:
        print('Game does not exist: ', url)

# convert resulting season-long lists into Dataframe
reg20212022_events_df = pd.DataFrame({
    'event_id': event_id,
    'game_id': game_id,
    'player1_id': player1_id,
    'player1_name': player1_name,
    'player1_link': player1_link,
    'player1_type': player1_type,
    'player2_id': player2_id,
    'player2_name': player2_name,
    'player2_link': player2_link,
    'player2_type': player2_type,
    'event': event,
    'eventCode': eventCode,
    'eventType': eventType,
    'eventDescription': eventDescription,
    'secondaryType': secondaryType,
    'primaryAssister_id': primaryAssister_id,
    'primaryAssister_name': primaryAssister_name,
    'primaryAssister_link': primaryAssister_link,
    'primaryAssister_type': primaryAssister_type,
    'primaryAssister_total': primaryAssister_total,
    'secondaryAssister_id': secondaryAssister_id,
    'secondaryAssister_name': secondaryAssister_name,
    'secondaryAssister_link': secondaryAssister_link,
    'secondaryAssister_type': secondaryAssister_type,
    'secondaryAssister_total': secondaryAssister_total,
    'penaltyServer_id': penaltyServer_id,
    'penaltyServer_name': penaltyServer_name,
    'penaltyServer_link': penaltyServer_link,
    'penaltyServer_type': penaltyServer_type,
    'strength': strength,
    'gwg': gwg,
    'emptyNet': emptyNet,
    'scorer_total': scorer_total,
    'penaltySeverity': penaltySeverity,
    'penaltyMinutes': penaltyMinutes,
    'eventIdx': eventIdx,
    'period': period,
    'periodType': periodType,
    'periodTime': periodTime,
    'dateTime': dateTime,
    'awayGoals': awayGoals,
    'homeGoals': homeGoals,
    'xCoord': xCoord,
    'yCoord': yCoord,
    'team1_id': team1_id,
    'team1_name': team1_name,
    'team1_link': team1_link,
    'team1_tricode': team1_tricode
})

In [ ]:
reg20212022_events_df = reg20202021_events_df.set_index('event_id')

In [ ]:
reg20212022_events_df[:10]

## convert to Glue-ready form

In [ ]:
from urllib.request import Request, urlopen
from io import StringIO
import boto3
import json
import pandas as pd
import requests

req = Request('https://statsapi.web.nhl.com/api/v1/game/2020020001/feed/live', 
              headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
page_json = json.loads(webpage.decode('utf-8'))

##########

# create lists to store events for the entire season, to be converted to Dataframe
event_id = []
game_id = []
player1_id = []
player1_name = []
player1_link = []
player1_type = []
player2_id = []
player2_name = []
player2_link = []
player2_type = []
primaryAssister_id = []
primaryAssister_name = []
primaryAssister_link = []
primaryAssister_type = []
primaryAssister_total = []
secondaryAssister_id = []
secondaryAssister_name = []
secondaryAssister_link = []
secondaryAssister_type = []
secondaryAssister_total = []
penaltyServer_id = []
penaltyServer_name = []
penaltyServer_link = []
penaltyServer_type = []
strength = []
gwg = []
emptyNet = []
scorer_total = []
event = []
eventCode = []
eventType = []
eventDescription = []
secondaryType = []
penaltySeverity = []
penaltyMinutes = []
eventIdx = []
period = []
periodType = []
periodTime = []
dateTime = []
awayGoals = []
homeGoals = []
xCoord = []
yCoord = []
team1_id = []
team1_name = []
team1_link = []
team1_tricode = []

root = '202102'
for game in range(1, 400):
    if game<10:
        code = root+'000'+str(game)
    elif game<100:
        code = root+'00'+str(game)
    elif game<1000:
        code = root+'0'+str(game)
    else:
        code = root+str(game)
        
            
    url = 'https://statsapi.web.nhl.com/api/v1/game/'+str(code)+'/feed/live'
    
    # access page and pull relevant JSON
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    
    # check if valid url
    response = requests.get(url)
    if response.status_code == 200:
        webpage = urlopen(req).read()
        page_json = json.loads(webpage)
        game_data = page_json['gameData']
        
        # check if game has been played
        if game_data['status']['abstractGameState'] == "Final":
            events = page_json['liveData']['plays']['allPlays']

            # iterate through all events and store as a row
            for i in range(len(events)):

                # add game_id to every row
                game_id.append(game_data['game']['pk'])

                # these keys exist for every event, whether hockey play or not
                event_result = events[i]['result']
                event_about = events[i]['about']

                # event is a hockey event (Shot, Goal, Hit, etc.)
                if 'players' in events[i]:

                    player_info = events[i]['players']

                    # event type is a Giveaway, Takeaway, or Missed Shot
                    if len(player_info) == 1:
                        player1_id.append(player_info[0]['player']['id'])
                        player1_name.append(player_info[0]['player']['fullName'])
                        player1_link.append(player_info[0]['player']['link'])
                        player1_type.append(player_info[0]['playerType'])
                        player2_id.append('N/A')
                        player2_name.append('N/A')
                        player2_link.append('N/A')
                        player2_type.append('N/A')
                        primaryAssister_id.append('N/A')
                        primaryAssister_name.append('N/A')
                        primaryAssister_link.append('N/A')
                        primaryAssister_type.append('N/A')
                        primaryAssister_total.append('N/A')
                        secondaryAssister_id.append('N/A')
                        secondaryAssister_name.append('N/A')
                        secondaryAssister_link.append('N/A')
                        secondaryAssister_type.append('N/A')
                        secondaryAssister_total.append('N/A')
                        penaltyServer_id.append('N/A')
                        penaltyServer_name.append('N/A')
                        penaltyServer_link.append('N/A')
                        penaltyServer_type.append('N/A')
                    # event type is not a Goal, Giveaway, Takeaway, or Missed Shot or it's a goal with no assists
                    elif len(player_info) == 2:
                        player1_id.append(player_info[0]['player']['id'])
                        player1_name.append(player_info[0]['player']['fullName'])
                        player1_link.append(player_info[0]['player']['link'])
                        player1_type.append(player_info[0]['playerType'])
                        player2_id.append(player_info[1]['player']['id'])
                        player2_name.append(player_info[1]['player']['fullName'])
                        player2_link.append(player_info[1]['player']['link'])
                        player2_type.append(player_info[1]['playerType'])   
                        primaryAssister_id.append('N/A')
                        primaryAssister_name.append('N/A')
                        primaryAssister_link.append('N/A')
                        primaryAssister_type.append('N/A')
                        primaryAssister_total.append('N/A')
                        secondaryAssister_id.append('N/A')
                        secondaryAssister_name.append('N/A')
                        secondaryAssister_link.append('N/A')
                        secondaryAssister_type.append('N/A')
                        secondaryAssister_total.append('N/A')
                        penaltyServer_id.append('N/A')
                        penaltyServer_name.append('N/A')
                        penaltyServer_link.append('N/A')
                        penaltyServer_type.append('N/A')
                    # event type is a Goal with one assist or a Penalty taken and served by different players
                    elif len(player_info) == 3:
                        player1_id.append(player_info[0]['player']['id'])
                        player1_name.append(player_info[0]['player']['fullName'])
                        player1_link.append(player_info[0]['player']['link'])
                        player1_type.append(player_info[0]['playerType'])
                        # event type is Goal
                        if event_result['eventTypeId'] == 'GOAL':
                            primaryAssister_id.append(player_info[1]['player']['id'])
                            primaryAssister_name.append(player_info[1]['player']['fullName'])
                            primaryAssister_link.append(player_info[1]['player']['link'])
                            primaryAssister_type.append(player_info[1]['playerType'])
                            primaryAssister_total.append(player_info[1]['seasonTotal'])
                            player2_id.append(player_info[2]['player']['id'])
                            player2_name.append(player_info[2]['player']['fullName'])
                            player2_link.append(player_info[2]['player']['link'])
                            player2_type.append(player_info[2]['playerType'])
                            penaltyServer_id.append('N/A')
                            penaltyServer_name.append('N/A')
                            penaltyServer_link.append('N/A')
                            penaltyServer_type.append('N/A')
                        # event type is Penalty
                        elif event_result['eventTypeId'] == 'PENALTY':
                            player2_id.append(player_info[1]['player']['id'])
                            player2_name.append(player_info[1]['player']['fullName'])
                            player2_link.append(player_info[1]['player']['link'])
                            player2_type.append(player_info[1]['playerType'])
                            penaltyServer_id.append(player_info[2]['player']['id'])
                            penaltyServer_name.append(player_info[2]['player']['fullName'])
                            penaltyServer_link.append(player_info[2]['player']['link'])
                            penaltyServer_type.append(player_info[2]['playerType'])
                            primaryAssister_id.append('N/A')
                            primaryAssister_name.append('N/A')
                            primaryAssister_link.append('N/A')
                            primaryAssister_type.append('N/A')
                            primaryAssister_total.append('N/A')
                        # who fucking knows
                        else:
                            print(game_data['game']['pk'])
                            
                        secondaryAssister_id.append('N/A')
                        secondaryAssister_name.append('N/A')
                        secondaryAssister_link.append('N/A')
                        secondaryAssister_type.append('N/A')
                        secondaryAssister_total.append('N/A')
                    # event type is a Goal with two assists
                    elif len(player_info) == 4:
                        player1_id.append(player_info[0]['player']['id'])
                        player1_name.append(player_info[0]['player']['fullName'])
                        player1_link.append(player_info[0]['player']['link'])
                        player1_type.append(player_info[0]['playerType'])
                        player2_id.append(player_info[3]['player']['id'])
                        player2_name.append(player_info[3]['player']['fullName'])
                        player2_link.append(player_info[3]['player']['link'])
                        player2_type.append(player_info[3]['playerType']) 
                        primaryAssister_id.append(player_info[1]['player']['id'])
                        primaryAssister_name.append(player_info[1]['player']['fullName'])
                        primaryAssister_link.append(player_info[1]['player']['link'])
                        primaryAssister_type.append(player_info[1]['playerType'])
                        primaryAssister_total.append(player_info[1]['seasonTotal'])
                        secondaryAssister_id.append(player_info[2]['player']['id'])
                        secondaryAssister_name.append(player_info[2]['player']['fullName'])
                        secondaryAssister_link.append(player_info[2]['player']['link'])
                        secondaryAssister_type.append(player_info[2]['playerType'])
                        secondaryAssister_total.append(player_info[2]['seasonTotal'])
                        penaltyServer_id.append('N/A')
                        penaltyServer_name.append('N/A')
                        penaltyServer_link.append('N/A')
                        penaltyServer_type.append('N/A')
                    else:
                        print('What the fuck')



                    # event is a Goal, Shot, or Penalty
                    if 'secondaryType' in event_result:
                        secondaryType.append(event_result['secondaryType'])
                    else:
                        secondaryType.append('N/A')



                    # event is a Penalty
                    if event_result['eventTypeId'] == 'PENALTY':
                        penaltySeverity.append(event_result['penaltySeverity'])
                        penaltyMinutes.append(event_result['penaltyMinutes'])
                    else:
                        penaltySeverity.append('N/A')
                        penaltyMinutes.append('N/A')



                    # event is a Goal
                    if event_result['eventTypeId'] == 'GOAL':
                        strength.append(event_result['strength']['code'])
                        gwg.append(event_result['gameWinningGoal'])
                        # not sure why but some goal events are missing emptyNet key
                        if 'emptyNet' in event_result:
                            emptyNet.append(event_result['emptyNet'])
                        else:
                            emptyNet.append('N/A')
                        scorer_total.append(player_info[0]['seasonTotal'])
                    else:
                        strength.append('N/A')
                        gwg.append('N/A')
                        emptyNet.append('N/A')
                        scorer_total.append('N/A')



                    eventIdx.append(event_about['eventIdx'])
                    event_id.append(str(game_data['game']['pk']) + '-' + str(event_about['eventIdx'])) # primary key
                    event.append(event_result['event'])
                    eventCode.append(event_result['eventCode'])
                    eventType.append(event_result['eventTypeId'])
                    eventDescription.append(event_result['description'])
                    period.append(event_about['period'])
                    periodType.append(event_about['periodType'])
                    periodTime.append(event_about['periodTime'])
                    dateTime.append(event_about['dateTime'])
                    awayGoals.append(event_about['goals']['away'])
                    homeGoals.append(event_about['goals']['home'])
                    
                    # some events are missing coordinates, no idea why
                    if events[i]['coordinates']:
                        xCoord.append(events[i]['coordinates']['x'])
                        yCoord.append(events[i]['coordinates']['y'])
                    else:
                        xCoord.append('Missing')
                        yCoord.append('Missing')
                        
                    team1_id.append(events[i]['team']['id'])
                    team1_name.append(events[i]['team']['name'])
                    team1_link.append(events[i]['team']['link'])
                    team1_tricode.append(events[i]['team']['triCode'])



                # event is a non-hockey event (Game start/end, Period start/end, Stoppage, etc.)
                else:
                    player1_id.append('N/A')
                    player1_name.append('N/A')
                    player1_link.append('N/A')
                    player1_type.append('N/A')
                    player2_id.append('N/A')
                    player2_name.append('N/A')
                    player2_link.append('N/A')
                    player2_type.append('N/A')
                    primaryAssister_id.append('N/A')
                    primaryAssister_name.append('N/A')
                    primaryAssister_link.append('N/A')
                    primaryAssister_type.append('N/A')
                    primaryAssister_total.append('N/A')
                    secondaryAssister_id.append('N/A')
                    secondaryAssister_name.append('N/A')
                    secondaryAssister_link.append('N/A')
                    secondaryAssister_type.append('N/A')
                    secondaryAssister_total.append('N/A')
                    penaltyServer_id.append('N/A')
                    penaltyServer_name.append('N/A')
                    penaltyServer_link.append('N/A')
                    penaltyServer_type.append('N/A')
                    strength.append('N/A')
                    gwg.append('N/A')
                    emptyNet.append('N/A')
                    scorer_total.append('N/A')
                    event.append(event_result['event'])
                    eventCode.append(event_result['eventCode'])
                    eventType.append(event_result['eventTypeId'])
                    eventDescription.append(event_result['description'])
                    secondaryType.append('N/A')
                    penaltySeverity.append('N/A')
                    penaltyMinutes.append('N/A')
                    eventIdx.append(event_about['eventIdx'])
                    event_id.append(str(game_data['game']['pk']) + '-' + str(event_about['eventIdx'])) # primary key
                    period.append(event_about['period'])
                    periodType.append(event_about['periodType'])
                    periodTime.append(event_about['periodTime'])
                    dateTime.append(event_about['dateTime'])
                    awayGoals.append(event_about['goals']['away'])
                    homeGoals.append(event_about['goals']['home'])
                    xCoord.append('N/A')
                    yCoord.append('N/A')
                    team1_id.append('N/A')
                    team1_name.append('N/A')
                    team1_link.append('N/A')
                    team1_tricode.append('N/A')
                    
        else:
            event_id.append('Not played')
            game_id.append(game_data['game']['pk'])
            player1_id.append('Not played')
            player1_name.append('Not played')
            player1_link.append('Not played')
            player1_type.append('Not played')
            player2_id.append('Not played')
            player2_name.append('Not played')
            player2_link.append('Not played')
            player2_type.append('Not played')
            primaryAssister_id.append('Not played')
            primaryAssister_name.append('Not played')
            primaryAssister_link.append('Not played')
            primaryAssister_type.append('Not played')
            primaryAssister_total.append('Not played')
            secondaryAssister_id.append('Not played')
            secondaryAssister_name.append('Not played')
            secondaryAssister_link.append('Not played')
            secondaryAssister_type.append('Not played')
            secondaryAssister_total.append('Not played')
            penaltyServer_id.append('Not played')
            penaltyServer_name.append('Not played')
            penaltyServer_link.append('Not played')
            penaltyServer_type.append('Not played')
            strength.append('Not played')
            gwg.append('Not played')
            emptyNet.append('Not played')
            scorer_total.append('Not played')
            event.append('Not played')
            eventCode.append('Not played')
            eventType.append('Not played')
            eventDescription.append('Not played')
            secondaryType.append('Not played')
            penaltySeverity.append('Not played')
            penaltyMinutes.append('Not played')
            eventIdx.append('Not played')
            period.append('Not played')
            periodType.append('Not played')
            periodTime.append('Not played')
            dateTime.append('Not played')
            awayGoals.append('Not played')
            homeGoals.append('Not played')
            xCoord.append('Not played')
            yCoord.append('Not played')
            team1_id.append('Not played')
            team1_name.append('Not played')
            team1_link.append('Not played')
            team1_tricode.append('Not played')

# convert resulting season-long lists into Dataframe
reg20212022_events_df = pd.DataFrame({
    'event_id': event_id,
    'game_id': game_id,
    'player1_id': player1_id,
    'player1_name': player1_name,
    'player1_link': player1_link,
    'player1_type': player1_type,
    'player2_id': player2_id,
    'player2_name': player2_name,
    'player2_link': player2_link,
    'player2_type': player2_type,
    'event': event,
    'eventCode': eventCode,
    'eventType': eventType,
    'eventDescription': eventDescription,
    'secondaryType': secondaryType,
    'primaryAssister_id': primaryAssister_id,
    'primaryAssister_name': primaryAssister_name,
    'primaryAssister_link': primaryAssister_link,
    'primaryAssister_type': primaryAssister_type,
    'primaryAssister_total': primaryAssister_total,
    'secondaryAssister_id': secondaryAssister_id,
    'secondaryAssister_name': secondaryAssister_name,
    'secondaryAssister_link': secondaryAssister_link,
    'secondaryAssister_type': secondaryAssister_type,
    'secondaryAssister_total': secondaryAssister_total,
    'penaltyServer_id': penaltyServer_id,
    'penaltyServer_name': penaltyServer_name,
    'penaltyServer_link': penaltyServer_link,
    'penaltyServer_type': penaltyServer_type,
    'strength': strength,
    'gwg': gwg,
    'emptyNet': emptyNet,
    'scorer_total': scorer_total,
    'penaltySeverity': penaltySeverity,
    'penaltyMinutes': penaltyMinutes,
    'eventIdx': eventIdx,
    'period': period,
    'periodType': periodType,
    'periodTime': periodTime,
    'dateTime': dateTime,
    'awayGoals': awayGoals,
    'homeGoals': homeGoals,
    'xCoord': xCoord,
    'yCoord': yCoord,
    'team1_id': team1_id,
    'team1_name': team1_name,
    'team1_link': team1_link,
    'team1_tricode': team1_tricode
})

##########

bucket = 'pipeline-glue-bucket-01'
csv_buffer = StringIO()
reg20212022_events_df.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'reg20212022_events_df.csv').put(Body=csv_buffer.getvalue())

# Analysis

In [ ]:
reg20202021_events_df['secondaryType'].value_counts()

# Scratch

In [ ]:
len(reg20202021_events_df)

In [ ]:
combined = pd.concat(reg20192020_events_df, reg20202021_events_df)

In [ ]:
# get dictionary of "gameData" section which contains game info
game_data = page_json['gameData']

# get abbreviations of home and away teams in game
home_team = game_data['teams']['home']['abbreviation']
away_team = game_data['teams']['away']['abbreviation']

# get dictionary of "boxscore" section which contains all players and shift times at different strengths
boxscore = page_json['liveData']['boxscore']

# get dictionary of "players" section
home_players = boxscore['teams']['home']['players']
away_players = boxscore['teams']['away']['players']

# get list of player IDs for each team
home_IDs = list(home_players.keys())
away_IDs = list(away_players.keys())